In [1]:
import pandas as pd
import numpy as np

In [2]:
pip install fsspec

In [3]:
pip install s3fs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 77.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
    Uninstalling fsspec-2023.6.0:
      Successfully uninstalled fsspec-2023.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2023.6.0 requires fsspec==2023.6.0, but you have fsspec 2023.10.0 which is incompatible.


## Preparing Meta Dataset

In [4]:
returns_gas_ind_target = pd.read_csv('s3://fintech540-ml/staging/transform/RETURNS_GAS_IND_TARGET.csv', parse_dates=['Timestamp'], index_col=['Timestamp'])
vix_close_prices_yahoo = pd.read_csv('s3://fintech540-ml/external/VIX_CLOSE_PRICES_YAHOO.csv', parse_dates=['Date'], index_col=['Date'])
dow_volume_yahoo = pd.read_csv('s3://fintech540-ml/external/dow_volume_yahoo.csv', parse_dates=['Date'], index_col=['Date'])
cpi_data = pd.read_csv('s3://fintech540-ml/external/cpi_data.csv', parse_dates=['Date'], index_col=['Date'])
sector_etf_data = pd.read_csv('s3://fintech540-ml/external/Five_sectors.csv', parse_dates=['Date'], index_col=['Date'])

<ipython-input-4-2895e7f69b93>:1: DtypeWarning: Columns (35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  returns_gas_ind_target = pd.read_csv('s3://fintech540-ml/staging/transform/RETURNS_GAS_IND_TARGET.csv', parse_dates=['Timestamp'], index_col=['Timestamp'])


In [9]:
pd.set_option('display.max_columns', None)

returns_gas_ind_target.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT,gas_fiscal_year,gas_fiscal_week,gas_current_year_production,gas_previous_year_production,gas_difference_from_same_week_last_year,gas_current_year_cumulative_production,gas_cumulative_difference,aerospace_funds_raised_mean,construction_funds_raised_mean,consumer_funds_raised_mean,crypto_funds_raised_mean,data_funds_raised_mean,education_funds_raised_mean,energy_funds_raised_mean,finance_funds_raised_mean,fitness_funds_raised_mean,food_funds_raised_mean,hardware_funds_raised_mean,healthcare_funds_raised_mean,hr_funds_raised_mean,infrastructure_funds_raised_mean,legal_funds_raised_mean,logistics_funds_raised_mean,manufacturing_funds_raised_mean,marketing_funds_raised_mean,media_funds_raised_mean,other_funds_raised_mean,product_funds_raised_mean,real estate_funds_raised_mean,recruiting_funds_raised_mean,retail_funds_raised_mean,sales_funds_raised_mean,security_funds_raised_mean,support_funds_raised_mean,transportation_funds_raised_mean,travel_funds_raised_mean,aerospace_funds_raised_sum,construction_funds_raised_sum,consumer_funds_raised_sum,crypto_funds_raised_sum,data_funds_raised_sum,education_funds_raised_sum,energy_funds_raised_sum,finance_funds_raised_sum,fitness_funds_raised_sum,food_funds_raised_sum,hardware_funds_raised_sum,healthcare_funds_raised_sum,hr_funds_raised_sum,infrastructure_funds_raised_sum,legal_funds_raised_sum,logistics_funds_raised_sum,manufacturing_funds_raised_sum,marketing_funds_raised_sum,media_funds_raised_sum,other_funds_raised_sum,product_funds_raised_sum,real estate_funds_raised_sum,recruiting_funds_raised_sum,retail_funds_raised_sum,sales_funds_raised_sum,security_funds_raised_sum,support_funds_raised_sum,transportation_funds_raised_sum,travel_funds_raised_sum,aerospace_laid_off_count_mean,construction_laid_off_count_mean,consumer_laid_off_count_mean,crypto_laid_off_count_mean,data_laid_off_count_mean,education_laid_off_count_mean,energy_laid_off_count_mean,finance_laid_off_count_mean,fitness_laid_off_count_mean,food_laid_off_count_mean,hardware_laid_off_count_mean,healthcare_laid_off_count_mean,hr_laid_off_count_mean,infrastructure_laid_off_count_mean,legal_laid_off_count_mean,logistics_laid_off_count_mean,manufacturing_laid_off_count_mean,marketing_laid_off_count_mean,media_laid_off_count_mean,other_laid_off_count_mean,product_laid_off_count_mean,real estate_laid_off_count_mean,recruiting_laid_off_count_mean,retail_laid_off_count_mean,sales_laid_off_count_mean,security_laid_off_count_mean,support_laid_off_count_mean,transportation_laid_off_count_mean,travel_laid_off_count_mean,aerospace_laid_off_count_sum,construction_laid_off_count_sum,consumer_laid_off_count_sum,crypto_laid_off_count_sum,data_laid_off_count_sum,education_laid_off_count_sum,energy_laid_off_count_sum,finance_laid_off_count_sum,fitness_laid_off_count_sum,food_laid_off_count_sum,hardware_laid_off_count_sum,healthcare_laid_off_count_sum,hr_laid_off_count_sum,infrastructure_laid_off_count_sum,legal_laid_off_count_sum,logistics_laid_off_count_sum,manufacturing_laid_off_count_sum,marketing_laid_off_count_sum,media_laid_off_count_sum,other_laid_off_count_sum,product_laid_off_count_sum,real estate_laid_off_count_sum,recruiting_laid_off_count_sum,retail_laid_off_count_sum,sales_laid_off_count_sum,security_laid_off_count_sum,support_laid_off_count_sum,transportation_laid_off_count_sum,travel_laid_off_count_sum,top_performing_sector_t+1
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,0.232077,0.512068,-0.357439,0.251284,0.502541,0.258538,-0.204633,0.414504,0.476082,-0.049685,0.140398,-0.292642,-0.161615,0.086566,0.480305,0.232193,-0.081978,-0.242136,-0.177634,0.277781,0.471628,2020.0,23.0,2.778006e+09,2.687

### Creating naive data with stock and target column to get a naive performance with logistic regression

In [10]:
features = returns_gas_ind_target.columns[:30]
features

Index(['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'DOW',
       'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM',
       'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'VZ', 'V', 'WBA', 'WMT'],
      dtype='object')

In [11]:
target = returns_gas_ind_target.columns[-1]
target


'top_performing_sector_t+1'

In [12]:
naive_data_df = returns_gas_ind_target[features]
naive_data_df.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,0.232077,0.512068,-0.357439,0.251284,0.502541,0.258538,-0.204633,0.414504,0.476082,-0.049685,0.140398,-0.292642,-0.161615,0.086566,0.480305,0.232193,-0.081978,-0.242136,-0.177634,0.277781,0.471628
2020-06-01 11:00:00,-0.119307,-0.257543,-0.191428,-0.695396,-0.127898,-0.250637,-0.600353,-0.322545,0.028758,-0.158995,-0.599754,-0.080273,-0.216569,-0.245903,-0.382590,-0.237041,-0.131694,-0.150246,0.082623,-0.289746,-0.385857,-0.074866,-0.177802,0.250451,-0.007171,-0.064217,-0.227855,-0.220650,-0.450690,0.258141
2020-06-01 11:30:00,0.035965,0.085042,-0.117401,-0.170571,0.064520,0.549231,0.000090,0.018054,-0.297096,-0.170187,-0.048178,0.294151,-0.121606,-0.012857,-0.052165,0.024003,-0.038574,0.098440,0.325589,0.010300,-0.119639,0.127600,-0.275097,0.380821,0.322643,0.549230,-0.135073,-0.048526,-0.476453,0.261348
2020-06-01 12:00:00,0.077664,0.055355,-0.239098,0.158690,0.107169,0.275448,0.173565,0.336529,-0.019885,0.217921,-0.041686,0.238296,-0.024207,0.115936,-0.095580,-0.058971,-0.167178,0.205120,0.186116,0.011203,0.179846,0.100331,0.387637,0.060735,0.181023,0.234440,-0.095161,-0.105345,0.195576,-0.063781
2020-06-01 12:30:00,0.082847,0.010435,-0.228037,-0.112772,-0.001121,0.488219,0.340259,0.232288,0.127909,0.070464,-0.248766,-0.070714,0.033729,0.130911,-0.056183,-0.014819,-0.237539,0.216553,0.100173,-0.095978,0.057320,0.178239,0.294669,-0.008836,-0.089703,-0.120394,-0.088005,0.290172,-0.027918,0.048462


In [13]:
naive_target_df = returns_gas_ind_target[target]
naive_target_df.head()

Timestamp
2020-06-01 10:30:00         Industrials
2020-06-01 11:00:00              Energy
2020-06-01 11:30:00    Consumer Staples
2020-06-01 12:00:00          Healthcare
2020-06-01 12:30:00              Energy
Name: top_performing_sector_t+1, dtype: object

In [15]:
naive_data = pd.merge(naive_data_df, naive_target_df, how='outer', left_index=True, right_index=True)
naive_data.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT,top_performing_sector_t+1
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,0.232077,0.512068,-0.357439,0.251284,0.502541,0.258538,-0.204633,0.414504,0.476082,-0.049685,0.140398,-0.292642,-0.161615,0.086566,0.480305,0.232193,-0.081978,-0.242136,-0.177634,0.277781,0.471628,Industrials
2020-06-01 11:00:00,-0.119307,-0.257543,-0.191428,-0.695396,-0.127898,-0.250637,-0.600353,-0.322545,0.028758,-0.158995,-0.599754,-0.080273,-0.216569,-0.245903,-0.382590,-0.237041,-0.131694,-0.150246,0.082623,-0.289746,-0.385857,-0.074866,-0.177802,0.250451,-0.007171,-0.064217,-0.227855,-0.220650,-0.450690,0.258141,Energy
2020-06-01 11:30:00,0.035965,0.085042,-0.117401,-0.170571,0.064520,0.549231,0.000090,0.018054,-0.297096,-0.170187,-0.048178,0.294151,-0.121606,-0.012857,-0.052165,0.024003,-0.038574,0.098440,0.325589,0.010300,-0.119639,0.127600,-0.275097,0.380821,0.322643,0.549230,-0.135073,-0.048526,-0.476453,0.261348,Consumer Staples
2020-06-01 12:00:00,0.077664,0.055355,-0.239098,0.158690,0.107169,0.275448,0.173565,0.336529,-0.019885,0.217921,-0.041686,0.238296,-0.024207,0.115936,-0.095580,-0.058971,-0.167178,0.205120,0.186116,0.011203,0.179846,0.100331,0.387637,0.060735,0.181023,0.234440,-0.095161,-0.105345,0.195576,-0.063781,Healthcare
2020-06-01 12:30:00,0.082847,0.010435,-0.228037,-0.112772,-0.001121,0.488219,0.340259,0.232288,0.127909,0.070464,-0.248766,-0.070714,0.033729,0.130911,-0.056183,-0.014819,-0.237539,0.216553,0.100173,-0.095978,0.057320,0.178239,0.294669,-0.008836,-0.089703,-0.120394,-0.088005,0.290172,-0.027918,0.048462,Energy


In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
naive_data['top_performing_sector_t+1'] = le.fit_transform(naive_data['top_performing_sector_t+1'].astype(str))
naive_data.head()

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT,top_performing_sector_t+1
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,0.232077,0.512068,-0.357439,0.251284,0.502541,0.258538,-0.204633,0.414504,0.476082,-0.049685,0.140398,-0.292642,-0.161615,0.086566,0.480305,0.232193,-0.081978,-0.242136,-0.177634,0.277781,0.471628,5
2020-06-01 11:00:00,-0.119307,-0.257543,-0.191428,-0.695396,-0.127898,-0.250637,-0.600353,-0.322545,0.028758,-0.158995,-0.599754,-0.080273,-0.216569,-0.245903,-0.382590,-0.237041,-0.131694,-0.150246,0.082623,-0.289746,-0.385857,-0.074866,-0.177802,0.250451,-0.007171,-0.064217,-0.227855,-0.220650,-0.450690,0.258141,2
2020-06-01 11:30:00,0.035965,0.085042,-0.117401,-0.170571,0.064520,0.549231,0.000090,0.018054,-0.297096,-0.170187,-0.048178,0.294151,-0.121606,-0.012857,-0.052165,0.024003,-0.038574,0.098440,0.325589,0.010300,-0.119639,0.127600,-0.275097,0.380821,0.322643,0.549230,-0.135073,-0.048526,-0.476453,0.261348,1
2020-06-01 12:00:00,0.077664,0.055355,-0.239098,0.158690,0.107169,0.275448,0.173565,0.336529,-0.019885,0.217921,-0.041686,0.238296,-0.024207,0.115936,-0.095580,-0.058971,-0.167178,0.205120,0.186116,0.011203,0.179846,0.100331,0.387637,0.060735,0.181023,0.234440,-0.095161,-0.105345,0.195576,-0.063781,4
2020-06-01 12:30:00,0.082847,0.010435,-0.228037,-0.112772,-0.001121,0.488219,0.340259,0.232288,0.127909,0.070464,-0.248766,-0.070714,0.033729,0.130911,-0.056183,-0.014819,-0.237539,0.216553,0.100173,-0.095978,0.057320,0.178239,0.294669,-0.008836,-0.089703,-0.120394,-0.088005,0.290172,-0.027918,0.048462,2


In [21]:
train_set = naive_data[:'2022-12-31']
test_set = naive_data['2023-01-01':]

X_train = train_set[features]
y_train = train_set[target]
X_test = test_set[features]
y_test = test_set[target]

In [22]:
#fit logistic regression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_scaled, y_train)

# Generate predictions on the test set
X_test_scaled = scaler.transform(X_test)
y_pred = log_reg.predict(X_test_scaled)  # Make sure X_test_combined is prepared similarly

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Test Accuracy: 0.44642857142857145
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        86
           1       0.00      0.00      0.00        86
           2       0.00      0.00      0.00       240
           3       0.00      0.00      0.00       106
           4       0.00      0.00      0.00       159
           5       0.00      0.00      0.00       106
           6       0.45      0.97      0.62       980
           7       0.00      0.00      0.00       165
           8       0.00      0.00      0.00       200

    accuracy                           0.45      2128
   macro avg       0.05      0.11      0.07      2128
weighted avg       0.21      0.45      0.28      2128



In [23]:
#fit xgboost classifier
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', device ='cuda')

#fit model
xgb.fit(X_train, y_train)

#predict
y_pred = xgb.predict(X_test)

#check accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.11      0.03      0.05        86
           1       0.16      0.03      0.06        86
           2       0.18      0.28      0.22       240
           3       0.20      0.02      0.03       106
           4       0.18      0.09      0.12       159
           5       0.07      0.03      0.04       106
           6       0.66      0.89      0.76       980
           7       0.17      0.11      0.13       165
           8       0.24      0.17      0.20       200

    accuracy                           0.48      2128
   macro avg       0.22      0.18      0.18      2128
weighted avg       0.40      0.48      0.42      2128



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:15:38] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [28]:
X_train[:-1]

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,0.232077,0.512068,-0.357439,0.251284,0.502541,0.258538,-0.204633,0.414504,0.476082,-0.049685,0.140398,-0.292642,-0.161615,0.086566,0.480305,0.232193,-0.081978,-0.242136,-0.177634,0.277781,0.471628
2020-06-01 11:00:00,-0.119307,-0.257543,-0.191428,-0.695396,-0.127898,-0.250637,-0.600353,-0.322545,0.028758,-0.158995,-0.599754,-0.080273,-0.216569,-0.245903,-0.382590,-0.237041,-0.131694,-0.150246,0.082623,-0.289746,-0.385857,-0.074866,-0.177802,0.250451,-0.007171,-0.064217,-0.227855,-0.220650,-0.450690,0.258141
2020-06-01 11:30:00,0.035965,0.085042,-0.117401,-0.170571,0.064520,0.549231,0.000090,0.018054,-0.297096,-0.170187,-0.048178,0.294151,-0.121606,-0.012857,-0.052165,0.024003,-0.038574,0.098440,0.325589,0.010300,-0.119639,0.127600,-0.275097,0.380821,0.322643,0.549230,-0.135073,-0.048526,-0.476453,0.261348
2020-06-01 12:00:00,0.077664,0.055355,-0.239098,0.158690,0.107169,0.275448,0.173565,0.336529,-0.019885,0.217921,-0.041686,0.238296,-0.024207,0.115936,-0.095580,-0.058971,-0.167178,0.205120,0.186116,0.011203,0.179846,0.100331,0.387637,0.060735,0.181023,0.234440,-0.095161,-0.105345,0.195576,-0.063781
2020-06-01 12:30:00,0.082847,0.010435,-0.228037,-0.112772,-0.001121,0.488219,0.340259,0.232288,0.127909,0.070464,-0.248766,-0.070714,0.033729,0.130911,-0.056183,-0.014819,-0.237539,0.216553,0.100173,-0.095978,0.057320,0.178239,0.294669,-0.008836,-0.089703,-0.120394,-0.088005,0.290172,-0.027918,0.048462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 13:30:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-12-31 14:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-12-31 14:30:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [29]:
pd.set_option('display.max_columns', None)
returns_gas_ind_target['date_idx'] = returns_gas_ind_target.index.strftime('%Y-%m-%d')
returns_gas_ind_target.insert(0, 'date_idx', returns_gas_ind_target.pop('date_idx'))
# pd.to_datetime(returns_gas_ind_target[']format='%Y-%m-%d')
returns_gas_ind_target = returns_gas_ind_target.reset_index()
returns_gas_ind_target['date_idx'] = pd.to_datetime(returns_gas_ind_target['date_idx'])
returns_gas_ind_target = returns_gas_ind_target.set_index('date_idx')
returns_gas_ind_target.head()

,Timestamp,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT,gas_fiscal_year,gas_fiscal_week,gas_current_year_production,gas_previous_year_production,gas_difference_from_same_week_last_year,gas_current_year_cumulative_production,gas_cumulative_difference,aerospace_funds_raised_mean,construction_funds_raised_mean,consumer_funds_raised_mean,crypto_funds_raised_mean,data_funds_raised_mean,education_funds_raised_mean,energy_funds_raised_mean,finance_funds_raised_mean,fitness_funds_raised_mean,food_funds_raised_mean,hardware_funds_raised_mean,healthcare_funds_raised_mean,hr_funds_raised_mean,infrastructure_funds_raised_mean,legal_funds_raised_mean,logistics_funds_raised_mean,manufacturing_funds_raised_mean,marketing_funds_raised_mean,media_funds_raised_mean,other_funds_raised_mean,product_funds_raised_mean,real estate_funds_raised_mean,recruiting_funds_raised_mean,retail_funds_raised_mean,sales_funds_raised_mean,security_funds_raised_mean,support_funds_raised_mean,transportation_funds_raised_mean,travel_funds_raised_mean,aerospace_funds_raised_sum,construction_funds_raised_sum,consumer_funds_raised_sum,crypto_funds_raised_sum,data_funds_raised_sum,education_funds_raised_sum,energy_funds_raised_sum,finance_funds_raised_sum,fitness_funds_raised_sum,food_funds_raised_sum,hardware_funds_raised_sum,healthcare_funds_raised_sum,hr_funds_raised_sum,infrastructure_funds_raised_sum,legal_funds_raised_sum,logistics_funds_raised_sum,manufacturing_funds_raised_sum,marketing_funds_raised_sum,media_funds_raised_sum,other_funds_raised_sum,product_funds_raised_sum,real estate_funds_raised_sum,recruiting_funds_raised_sum,retail_funds_raised_sum,sales_funds_raised_sum,security_funds_raised_sum,support_funds_raised_sum,transportation_funds_raised_sum,travel_funds_raised_sum,aerospace_laid_off_count_mean,construction_laid_off_count_mean,consumer_laid_off_count_mean,crypto_laid_off_count_mean,data_laid_off_count_mean,education_laid_off_count_mean,energy_laid_off_count_mean,finance_laid_off_count_mean,fitness_laid_off_count_mean,food_laid_off_count_mean,hardware_laid_off_count_mean,healthcare_laid_off_count_mean,hr_laid_off_count_mean,infrastructure_laid_off_count_mean,legal_laid_off_count_mean,logistics_laid_off_count_mean,manufacturing_laid_off_count_mean,marketing_laid_off_count_mean,media_laid_off_count_mean,other_laid_off_count_mean,product_laid_off_count_mean,real estate_laid_off_count_mean,recruiting_laid_off_count_mean,retail_laid_off_count_mean,sales_laid_off_count_mean,security_laid_off_count_mean,support_laid_off_count_mean,transportation_laid_off_count_mean,travel_laid_off_count_mean,aerospace_laid_off_count_sum,construction_laid_off_count_sum,consumer_laid_off_count_sum,crypto_laid_off_count_sum,data_laid_off_count_sum,education_laid_off_count_sum,energy_laid_off_count_sum,finance_laid_off_count_sum,fitness_laid_off_count_sum,food_laid_off_count_sum,hardware_laid_off_count_sum,healthcare_laid_off_count_sum,hr_laid_off_count_sum,infrastructure_laid_off_count_sum,legal_laid_off_count_sum,logistics_laid_off_count_sum,manufacturing_laid_off_count_sum,marketing_laid_off_count_sum,media_laid_off_count_sum,other_laid_off_count_sum,product_laid_off_count_sum,real estate_laid_off_count_sum,recruiting_laid_off_count_sum,retail_laid_off_count_sum,sales_laid_off_count_sum,security_laid_off_count_sum,support_laid_off_count_sum,transportation_laid_off_count_sum,travel_laid_off_count_sum,top_performing_sector_t+1
date_idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-01,2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,0.232077,0.512068,-0.357439,0.251284,0.502541,0.258538,-0.204633,0.414504,0.476082,-0.049685,0.140398,-0.292642,-0.161615,0.086566,0.480305,0.232193,-0.081978,-0.242136,-0.177634,0.277781,0.471628,2020.0,23

In [30]:
vix_close_prices_yahoo = vix_close_prices_yahoo.rename(columns={'vs_previous': 'VIX_returns_pct'})
vix_close_prices_yahoo.drop(columns=['Close'], inplace=True)
vix_close_prices_yahoo.head()

,VIX_returns_pct
Date,
2020-06-01,0.026172
2020-06-02,-0.049238
2020-06-03,-0.043964
2020-06-04,0.005846
2020-06-05,-0.049981


In [31]:
dow_volume_yahoo = dow_volume_yahoo.rename(columns={'vs_previous': 'DJA_volume_pct'})
# dow_volume_yahoo.drop(columns=['Volume'], inplace=True)
dow_volume_yahoo.head()

,Volume,DJA_volume_pct
Date,,
2020-06-01,338310000,-0.358800
2020-06-02,352610000,0.042269
2020-06-03,398790000,0.130966
2020-06-04,372190000,-0.066702
2020-06-05,542310000,0.457078


In [32]:
cpi_data = cpi_data.rename(columns={'vs_previous': 'CPI_change_pct'})
# cpi_data.drop(columns=['CPI'], inplace=True)
cpi_data.head()

,CPI,CPI_change_pct
Date,,
2020-06-01,257.797,0.005472
2020-07-01,259.101,0.005058
2020-08-01,259.918,0.003153
2020-09-01,260.280,0.001393
2020-10-01,260.388,0.000415


In [33]:
# JOINS
example = pd.merge(vix_close_prices_yahoo, dow_volume_yahoo, how='outer', left_index=True, right_index=True)

In [34]:
example.head()

,VIX_returns_pct,Volume,DJA_volume_pct
Date,,,
2020-06-01,0.026172,338310000,-0.358800
2020-06-02,-0.049238,352610000,0.042269
2020-06-03,-0.043964,398790000,0.130966
2020-06-04,0.005846,372190000,-0.066702
2020-06-05,-0.049981,542310000,0.457078


In [35]:
example_etf_merge = pd.merge(example, sector_etf_data, how='outer', left_index=True, right_index=True)
example_etf_merge.head()

,VIX_returns_pct,Volume,DJA_volume_pct,XLK_ret,XLI_ret,XLV_ret,XLE_ret,XLF_ret
Date,,,,,,,,
2020-06-01,0.026172,338310000,-0.358800,NaN,NaN,NaN,NaN,NaN
2020-06-02,-0.049238,352610000,0.042269,0.0094,0.0130,0.0058,0.0279,0.0080
2020-06-03,-0.043964,398790000,0.130966,0.0078,0.0390,-0.0019,0.0294,0.0373
2020-06-04,0.005846,372190000,-0.066702,-0.0082,0.0106,-0.0079,0.0002,0.0218
2020-06-05,-0.049981,542310000,0.457078,0.0263,0.0367,0.0161,0.0743,0.0372


In [36]:
example_2 = pd.merge(example_etf_merge, cpi_data, how='outer', left_index=True, right_index=True)
example_2['CPI'] = example_2['CPI'].fillna(method='ffill')
example_2['CPI_change_pct'] = example_2['CPI_change_pct'].fillna(method='ffill')

In [37]:
# TABLE BELOW IS EXCELLENT
# pd.set_option('display.max_rows', None)
example_2.head(15)

,VIX_returns_pct,Volume,DJA_volume_pct,XLK_ret,XLI_ret,XLV_ret,XLE_ret,XLF_ret,CPI,CPI_change_pct
Date,,,,,,,,,,
2020-06-01,0.026172,338310000.0,-0.358800,NaN,NaN,NaN,NaN,NaN,257.797,0.005472
2020-06-02,-0.049238,352610000.0,0.042269,0.0094,0.0130,0.0058,0.0279,0.0080,257.797,0.005472
2020-06-03,-0.043964,398790000.0,0.130966,0.0078,0.0390,-0.0019,0.0294,0.0373,257.797,0.005472
2020-06-04,0.005846,372190000.0,-0.066702,-0.0082,0.0106,-0.0079,0.0002,0.0218,257.797,0.005472
2020-06-05,-0.049981,542310000.0,0.457078,0.0263,0.0367,0.0161,0.0743,0.0372,257.797,0.005472
2020-06-08,0.052610,430200000.0,-0.206727,0.0050,0.0201,0.0069,0.0450,0.0191,257.797,0.005472
2020-06-09,0.068191,381430000.0,-0.113366,0.0047,-0.0249,-0.0104,-0.0376,-0.0213,257.797,0.005472
2020-06-10,0.000000,448930000.0,0.176966,0.0167,-0.0239,-0.0016,-0.0492,-0.0374,257.797,0.005472
2020-06-11,0.479507,647780000.0,0.442942,-0.0573,-0.0702,-0.0556,-0.0942,-0.0818,257.797,0.005472


In [38]:
final_merge = pd.merge(returns_gas_ind_target, example_2, how='outer', left_index=True, right_index=True)

In [39]:
final_merge = final_merge.set_index('Timestamp')
final_merge.head(50)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT,gas_fiscal_year,gas_fiscal_week,gas_current_year_production,gas_previous_year_production,gas_difference_from_same_week_last_year,gas_current_year_cumulative_production,gas_cumulative_difference,aerospace_funds_raised_mean,construction_funds_raised_mean,consumer_funds_raised_mean,crypto_funds_raised_mean,data_funds_raised_mean,education_funds_raised_mean,energy_funds_raised_mean,finance_funds_raised_mean,fitness_funds_raised_mean,food_funds_raised_mean,hardware_funds_raised_mean,healthcare_funds_raised_mean,hr_funds_raised_mean,infrastructure_funds_raised_mean,legal_funds_raised_mean,logistics_funds_raised_mean,manufacturing_funds_raised_mean,marketing_funds_raised_mean,media_funds_raised_mean,other_funds_raised_mean,product_funds_raised_mean,real estate_funds_raised_mean,recruiting_funds_raised_mean,retail_funds_raised_mean,sales_funds_raised_mean,security_funds_raised_mean,support_funds_raised_mean,transportation_funds_raised_mean,travel_funds_raised_mean,aerospace_funds_raised_sum,construction_funds_raised_sum,consumer_funds_raised_sum,crypto_funds_raised_sum,data_funds_raised_sum,education_funds_raised_sum,energy_funds_raised_sum,finance_funds_raised_sum,fitness_funds_raised_sum,food_funds_raised_sum,hardware_funds_raised_sum,healthcare_funds_raised_sum,hr_funds_raised_sum,infrastructure_funds_raised_sum,legal_funds_raised_sum,logistics_funds_raised_sum,manufacturing_funds_raised_sum,marketing_funds_raised_sum,media_funds_raised_sum,other_funds_raised_sum,product_funds_raised_sum,real estate_funds_raised_sum,recruiting_funds_raised_sum,retail_funds_raised_sum,sales_funds_raised_sum,security_funds_raised_sum,support_funds_raised_sum,transportation_funds_raised_sum,travel_funds_raised_sum,aerospace_laid_off_count_mean,construction_laid_off_count_mean,consumer_laid_off_count_mean,crypto_laid_off_count_mean,data_laid_off_count_mean,education_laid_off_count_mean,energy_laid_off_count_mean,finance_laid_off_count_mean,fitness_laid_off_count_mean,food_laid_off_count_mean,hardware_laid_off_count_mean,healthcare_laid_off_count_mean,hr_laid_off_count_mean,infrastructure_laid_off_count_mean,legal_laid_off_count_mean,logistics_laid_off_count_mean,manufacturing_laid_off_count_mean,marketing_laid_off_count_mean,media_laid_off_count_mean,other_laid_off_count_mean,product_laid_off_count_mean,real estate_laid_off_count_mean,recruiting_laid_off_count_mean,retail_laid_off_count_mean,sales_laid_off_count_mean,security_laid_off_count_mean,support_laid_off_count_mean,transportation_laid_off_count_mean,travel_laid_off_count_mean,aerospace_laid_off_count_sum,construction_laid_off_count_sum,consumer_laid_off_count_sum,crypto_laid_off_count_sum,data_laid_off_count_sum,education_laid_off_count_sum,energy_laid_off_count_sum,finance_laid_off_count_sum,fitness_laid_off_count_sum,food_laid_off_count_sum,hardware_laid_off_count_sum,healthcare_laid_off_count_sum,hr_laid_off_count_sum,infrastructure_laid_off_count_sum,legal_laid_off_count_sum,logistics_laid_off_count_sum,manufacturing_laid_off_count_sum,marketing_laid_off_count_sum,media_laid_off_count_sum,other_laid_off_count_sum,product_laid_off_count_sum,real estate_laid_off_count_sum,recruiting_laid_off_count_sum,retail_laid_off_count_sum,sales_laid_off_count_sum,security_laid_off_count_sum,support_laid_off_count_sum,transportation_laid_off_count_sum,travel_laid_off_count_sum,top_performing_sector_t+1,VIX_returns_pct,Volume,DJA_volume_pct,XLK_ret,XLI_ret,XLV_ret,XLE_ret,XLF_ret,CPI,CPI_change_pct
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,0.232077,0.512068,-0.357439,0.251284,0.502541,0.258538,-0.204633,0.414504,0.476082,-0.049685,0.140398,-0.292642,-0.16161

In [40]:
#find info about datatypes
final_merge.info()
#find how many total rows
final_merge.count()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15342 entries, 2020-06-01 10:30:00 to 2023-06-01 16:00:00
Columns: 164 entries, AAPL to CPI_change_pct
dtypes: float64(161), object(3)
memory usage: 19.3+ MB


AAPL              15342
AMGN              15342
AXP               15342
BA                15342
CAT               15342
                  ...  
XLV_ret           10570
XLE_ret           10570
XLF_ret           10570
CPI               10736
CPI_change_pct    10736
Length: 164, dtype: int64

In [41]:
#how many rows with null values or missing data
final_merge.isnull().sum()
#show rows with null values


AAPL                 0
AMGN                 0
AXP                  0
BA                   0
CAT                  0
                  ... 
XLV_ret           4772
XLE_ret           4772
XLF_ret           4772
CPI               4606
CPI_change_pct    4606
Length: 164, dtype: int64

In [42]:
objs = []
#total nan values in each column
for col in final_merge.columns:
    #check datatyep of each column
    if final_merge[col].dtype != 'float64':
        print(col)
        print(final_merge[col].dtype)
        objs.append(col)


gas_difference_from_same_week_last_year
object
gas_cumulative_difference
object
top_performing_sector_t+1
object


In [43]:
final_merge[objs]

,gas_difference_from_same_week_last_year,gas_cumulative_difference,top_performing_sector_t+1
Timestamp,,,
2020-06-01 10:30:00,90846000,280770000,Industrials
2020-06-01 11:00:00,90846000,280770000,Energy
2020-06-01 11:30:00,90846000,280770000,Consumer Staples
2020-06-01 12:00:00,90846000,280770000,Healthcare
2020-06-01 12:30:00,90846000,280770000,Energy
...,...,...,...
2023-06-01 14:00:00,NaN,NaN,Energy
2023-06-01 14:30:00,NaN,NaN,Technology
2023-06-01 15:00:00,NaN,NaN,Healthcare


In [44]:
final_merge['top_performing_sector_t+1'].unique()

array(['Industrials', 'Energy', 'Consumer Staples', 'Healthcare',
       'Telecommunications', 'Financials', 'Materials',
       'Consumer Discretionary', 'Technology'], dtype=object)

In [45]:
final_merge['gas_cumulative_difference'] = final_merge['gas_cumulative_difference'].apply(lambda x: float(x.replace("'", "")) if isinstance(x, str) else x)


In [46]:
final_merge['gas_difference_from_same_week_last_year'] = final_merge['gas_difference_from_same_week_last_year'].apply(lambda x: float(x.replace("'", "")) if isinstance(x, str) else x)

In [47]:
print(final_merge['gas_difference_from_same_week_last_year'].unique())

[ 9.084600e+07  8.437800e+07 -8.437800e+07 -7.267680e+08 -1.393854e+09
 -1.275666e+09 -1.204812e+09 -9.901920e+08 -9.428580e+08 -5.145000e+08
 -7.758660e+08 -6.294540e+08 -5.562480e+08 -5.812380e+08 -6.050520e+08
 -2.522520e+08 -2.737140e+08 -2.904720e+08 -1.664040e+08 -3.301620e+08
 -2.205000e+08 -3.039960e+08 -3.084060e+08 -2.928240e+08 -2.172660e+08
 -2.013900e+08 -4.165980e+08 -1.355340e+08 -2.443140e+08 -1.787520e+08
 -1.658160e+08 -2.287320e+08 -3.824940e+08 -3.642660e+08 -2.378460e+08
 -1.643460e+08 -2.745960e+08 -3.160500e+08 -3.113460e+08 -3.769080e+08
 -4.221840e+08 -3.766140e+08 -2.449020e+08 -2.034480e+08 -3.016440e+08
 -1.617000e+08 -2.822400e+08 -3.419220e+08 -2.543100e+08 -1.502340e+08
 -5.374320e+08 -3.051720e+08 -2.125620e+08 -3.686760e+08 -6.497400e+07
  6.562080e+08  1.092504e+09  1.135722e+09  1.115142e+09  8.869980e+08
  6.468000e+08  4.121880e+08  7.155960e+08  6.544440e+08  4.695180e+08
  1.705200e+08  4.380600e+08  2.446080e+08  1.799280e+08  3.754380e+08
  1.86

In [48]:
print(final_merge['gas_cumulative_difference'].unique())

[ 2.8077000e+08  3.6514800e+08 -4.4599800e+08 -1.8398520e+09
 -3.1155180e+09 -4.3203300e+09 -5.3105220e+09 -6.2533800e+09
 -6.7678800e+09 -7.5437460e+09 -8.1732000e+09 -8.7294480e+09
 -9.3106860e+09 -9.9157380e+09 -1.0167990e+10 -1.0441704e+10
 -1.0732176e+10 -1.0898580e+10 -1.1228742e+10 -1.1449242e+10
 -1.1753238e+10 -1.2061644e+10 -1.2354468e+10 -1.2571734e+10
 -1.2773124e+10 -1.3189722e+10 -1.3325256e+10 -1.3569570e+10
 -1.3748322e+10 -1.6581600e+08 -3.9454800e+08 -7.7704200e+08
 -1.1413080e+09 -1.3791540e+09 -1.5435000e+09 -1.8180960e+09
 -2.1341460e+09 -2.4454920e+09 -2.8224000e+09 -3.2445840e+09
 -3.6211980e+09 -3.8661000e+09 -4.0695480e+09 -4.3711920e+09
 -4.5328920e+09 -4.8151320e+09 -5.1570540e+09 -5.4113640e+09
 -5.5615980e+09 -6.0990300e+09 -6.4042020e+09 -6.6167640e+09
 -6.9854400e+09 -7.0504140e+09 -6.3942060e+09 -5.3017020e+09
 -4.1659800e+09 -3.0508380e+09 -2.1638400e+09 -1.5170400e+09
 -1.1048520e+09 -3.8925600e+08  2.6518800e+08  7.3470600e+08
  9.0522600e+08  1.34328

In [49]:
#convert to float
final_merge['gas_cumulative_difference'] = final_merge['gas_cumulative_difference'].astype(float)
final_merge['gas_difference_from_same_week_last_year'] = final_merge['gas_difference_from_same_week_last_year'].astype(float)

In [50]:
#find nan percentage of each column
for col in final_merge.columns:
    print(col)
    print(final_merge[col].isnull().sum()/len(final_merge[col]))

AAPL
0.0
AMGN
0.0
AXP
0.0
BA
0.0
CAT
0.0
CRM
0.0
CSCO
0.0
CVX
0.0
DIS
0.0
DOW
0.0
GS
0.0
HD
0.0
HON
0.0
IBM
0.0
INTC
0.0
JNJ
0.0
JPM
0.0
KO
0.0
MCD
0.0
MMM
0.0
MRK
0.0
MSFT
0.0
NKE
0.0
PG
0.0
TRV
0.0
UNH
0.0
VZ
0.0
V
0.0
WBA
0.0
WMT
0.0
gas_fiscal_year
0.04197627427975492
gas_fiscal_week
0.04197627427975492
gas_current_year_production
0.04197627427975492
gas_previous_year_production
0.04197627427975492
gas_difference_from_same_week_last_year
0.04836396819189154
gas_current_year_cumulative_production
0.04197627427975492
gas_cumulative_difference
0.04197627427975492
aerospace_funds_raised_mean
0.0
construction_funds_raised_mean
0.0
consumer_funds_raised_mean
0.0
crypto_funds_raised_mean
0.0
data_funds_raised_mean
0.0
education_funds_raised_mean
0.0
energy_funds_raised_mean
0.0
finance_funds_raised_mean
0.0
fitness_funds_raised_mean
0.0
food_funds_raised_mean
0.0
hardware_funds_raised_mean
0.0
healthcare_funds_raised_mean
0.0
hr_funds_raised_mean
0.0
infrastructure_funds_raised_mean
0.0
l

In [51]:
#drop gas difference from same week last year because it has 50% nan values
final_merge.drop(['gas_difference_from_same_week_last_year'], axis=1, inplace=True)


In [52]:
#show CPI rows where CPI is null
final_merge[final_merge['CPI'].isnull()]

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT,gas_fiscal_year,gas_fiscal_week,gas_current_year_production,gas_previous_year_production,gas_current_year_cumulative_production,gas_cumulative_difference,aerospace_funds_raised_mean,construction_funds_raised_mean,consumer_funds_raised_mean,crypto_funds_raised_mean,data_funds_raised_mean,education_funds_raised_mean,energy_funds_raised_mean,finance_funds_raised_mean,fitness_funds_raised_mean,food_funds_raised_mean,hardware_funds_raised_mean,healthcare_funds_raised_mean,hr_funds_raised_mean,infrastructure_funds_raised_mean,legal_funds_raised_mean,logistics_funds_raised_mean,manufacturing_funds_raised_mean,marketing_funds_raised_mean,media_funds_raised_mean,other_funds_raised_mean,product_funds_raised_mean,real estate_funds_raised_mean,recruiting_funds_raised_mean,retail_funds_raised_mean,sales_funds_raised_mean,security_funds_raised_mean,support_funds_raised_mean,transportation_funds_raised_mean,travel_funds_raised_mean,aerospace_funds_raised_sum,construction_funds_raised_sum,consumer_funds_raised_sum,crypto_funds_raised_sum,data_funds_raised_sum,education_funds_raised_sum,energy_funds_raised_sum,finance_funds_raised_sum,fitness_funds_raised_sum,food_funds_raised_sum,hardware_funds_raised_sum,healthcare_funds_raised_sum,hr_funds_raised_sum,infrastructure_funds_raised_sum,legal_funds_raised_sum,logistics_funds_raised_sum,manufacturing_funds_raised_sum,marketing_funds_raised_sum,media_funds_raised_sum,other_funds_raised_sum,product_funds_raised_sum,real estate_funds_raised_sum,recruiting_funds_raised_sum,retail_funds_raised_sum,sales_funds_raised_sum,security_funds_raised_sum,support_funds_raised_sum,transportation_funds_raised_sum,travel_funds_raised_sum,aerospace_laid_off_count_mean,construction_laid_off_count_mean,consumer_laid_off_count_mean,crypto_laid_off_count_mean,data_laid_off_count_mean,education_laid_off_count_mean,energy_laid_off_count_mean,finance_laid_off_count_mean,fitness_laid_off_count_mean,food_laid_off_count_mean,hardware_laid_off_count_mean,healthcare_laid_off_count_mean,hr_laid_off_count_mean,infrastructure_laid_off_count_mean,legal_laid_off_count_mean,logistics_laid_off_count_mean,manufacturing_laid_off_count_mean,marketing_laid_off_count_mean,media_laid_off_count_mean,other_laid_off_count_mean,product_laid_off_count_mean,real estate_laid_off_count_mean,recruiting_laid_off_count_mean,retail_laid_off_count_mean,sales_laid_off_count_mean,security_laid_off_count_mean,support_laid_off_count_mean,transportation_laid_off_count_mean,travel_laid_off_count_mean,aerospace_laid_off_count_sum,construction_laid_off_count_sum,consumer_laid_off_count_sum,crypto_laid_off_count_sum,data_laid_off_count_sum,education_laid_off_count_sum,energy_laid_off_count_sum,finance_laid_off_count_sum,fitness_laid_off_count_sum,food_laid_off_count_sum,hardware_laid_off_count_sum,healthcare_laid_off_count_sum,hr_laid_off_count_sum,infrastructure_laid_off_count_sum,legal_laid_off_count_sum,logistics_laid_off_count_sum,manufacturing_laid_off_count_sum,marketing_laid_off_count_sum,media_laid_off_count_sum,other_laid_off_count_sum,product_laid_off_count_sum,real estate_laid_off_count_sum,recruiting_laid_off_count_sum,retail_laid_off_count_sum,sales_laid_off_count_sum,security_laid_off_count_sum,support_laid_off_count_sum,transportation_laid_off_count_sum,travel_laid_off_count_sum,top_performing_sector_t+1,VIX_returns_pct,Volume,DJA_volume_pct,XLK_ret,XLI_ret,XLV_ret,XLE_ret,XLF_ret,CPI,CPI_change_pct
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-06 09:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020.0,23.0,2.778006e+09,2.687160e+09,6.145423e+10,280770000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [53]:
final_merge['top_performing_sector_t+1'].unique()

array(['Industrials', 'Energy', 'Consumer Staples', 'Healthcare',
       'Telecommunications', 'Financials', 'Materials',
       'Consumer Discretionary', 'Technology'], dtype=object)

## Modeling

### Naive Model No Tuning on All Data

In [140]:
#label encode top performing sector
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
final_merge['top_performing_sector_t+1'] = le.fit_transform(final_merge['top_performing_sector_t+1'].astype(str))

final_merge['top_performing_sector_t+1'].unique()

array([5, 2, 1, 4, 8, 3, 6, 0, 7])

In [141]:
#earliest date and latest date
print(final_merge.index.min())
print(final_merge.index.max())

2020-06-01 10:30:00
2023-06-01 16:00:00


In [142]:
#import train test split
from sklearn.model_selection import train_test_split

#split based on date index
#test be everything after 2022 and train be everything before 2022
train = final_merge[:'2022-12-31']
test = final_merge['2023-01-01':]

print(train.shape)
print(test.shape)

(13214, 163)
(2128, 163)


In [143]:
y_train = train['top_performing_sector_t+1']
y_test = test['top_performing_sector_t+1']

In [144]:
x_train = train.drop(['top_performing_sector_t+1'], axis=1)
x_test = test.drop(['top_performing_sector_t+1'], axis=1)

#check shape of train and test
print(x_train.shape)
print(x_test.shape)

(13214, 162)
(2128, 162)


In [145]:
#set up xgboostClassifier
from xgboost import XGBClassifier
xgb = XGBClassifier()

#fit model
xgb.fit(x_train, y_train)

#predict
y_pred = xgb.predict(x_test)

#check accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)


0.475093984962406

In [146]:
#score report
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.19      0.03      0.06        86
           1       0.08      0.01      0.02        86
           2       0.19      0.38      0.25       240
           3       0.00      0.00      0.00       106
           4       0.20      0.06      0.10       159
           5       0.12      0.07      0.08       106
           6       0.67      0.87      0.76       980
           7       0.17      0.11      0.13       165
           8       0.22      0.12      0.16       200

    accuracy                           0.48      2128
   macro avg       0.20      0.19      0.17      2128
weighted avg       0.40      0.48      0.42      2128



Use macro avg f1 score as benchmark due to the class imbalance. Macro avg treats all classes equally, while weighted avg taken into account the support for each class

In [147]:
#check for class imbalance
final_merge['top_performing_sector_t+1'].value_counts()

6    7333
2    1880
8    1221
7    1075
4    1028
5     811
0     709
3     649
1     636
Name: top_performing_sector_t+1, dtype: int64

In [148]:
#what are the sector mappings
le.inverse_transform([0, 1, 2, 3, 4, 5, 6, 7, 8])

array(['0', '1', '2', '3', '4', '5', '6', '7', '8'], dtype=object)

In [149]:
#total count for all sectors
final_merge['top_performing_sector_t+1'].value_counts().sum()

15342

In [150]:
#Naive performance metric (F1 weighted average of the classes) (macro average)
naive_performance_metric = (7333/15342)/9
print(naive_performance_metric)

0.053107663784237896


In [151]:
#random guess performance metric
random_guess_performance_metric = 1/9
print(random_guess_performance_metric)

0.1111111111111111


In [152]:
#compare model performance to naive performance

#retrive the macro average f1 score from the classification report
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred, output_dict=True)
macro_average_f1 = report['macro avg']['f1-score']
print(macro_average_f1)

0.17342203422867913


In [153]:
#get the performance improvement
performance_improvement = (macro_average_f1 - naive_performance_metric)/naive_performance_metric
print(performance_improvement)

2.2654803821393097


In [154]:
#performance improvement over random guess
performance_improvement_over_random_guess = (macro_average_f1 - random_guess_performance_metric)/random_guess_performance_metric
print(performance_improvement_over_random_guess)

0.5607983080581123


Basic Naive XGBOSST Classifier is better than the naive benchmarks

## Apply Tuning

In [155]:
#import libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

#import randomsearchcv
from sklearn.model_selection import RandomizedSearchCV
#import pipeline
from sklearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_sample_weight


In [ ]:
from scipy.stats import loguniform
# Define the XGBRegressor with the proper device specification
#to use without gpu remove tree_method='hist', device='cuda'
xgb_clf = XGBClassifier(tree_method='hist', device='cuda')

pipe_xgb = Pipeline([('xgb', xgb_clf)])
# Updated parameter grid with logarithmic scale and additional parameters
param_grid = {
    'xgb__n_estimators': [200, 300, 400, 500, 600],
    'xgb__max_depth': [2, 4, 6, 10, 15],
    'xgb__learning_rate': loguniform(0.01, 0.2), # Log scale
    'xgb__min_child_weight': [1, 2, 3],
    'xgb__reg_alpha': loguniform(1e-5, 1.0), # Log scale
    'xgb__reg_lambda': loguniform(1e-5, 1.0), # Log scale
    'xgb__gamma': [0.0, 1, 3, 6],
    'xgb__colsample_bytree': [0.4, 0.5, 0.7, 0.9],
    'xgb__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1]
}


rs_xgb = RandomizedSearchCV(pipe_xgb, param_grid, n_jobs=-1, random_state=42, n_iter=50, scoring='f1_macro')

# Fit model
rs_xgb.fit(x_train, y_train)

# Print best parameters and highest score
print('Best parameters found:\n', rs_xgb.best_params_)
print('Best F1 score:\n', rs_xgb.best_score_)
print('Best estimator:\n', rs_xgb.best_estimator_)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Tuning with RandomSearchCV did not improve macro f1

In [ ]:
#save pck file
#save model parameters
import json
params = rs_xgb.best_params_
with open('xgb_model_params.json', 'w') as fp:
    json.dump(params, fp)

#load model parameters
with open('xgb_model_params.json', 'r') as fp:
    params = json.load(fp)

#set params
xgb_clf = XGBClassifier(**params)

#fit the model
xgb_clf.fit(x_train, y_train)

# Generate predictions on the test set
y_test_predictions = xgb_clf.predict(x_test)

# Evaluate the predictions

accuracy = accuracy_score(y_test, y_test_predictions)
print("Test Accuracy:", accuracy)
print(classification_report(y_test, y_test_predictions))




## Creating Dataset that removes any columns with NaN

### Clean up NaN Columns

In [70]:
#show columns with more than 10% nan values
cols= []
for col in final_merge.columns:
    if final_merge[col].isnull().sum()/len(final_merge[col]) > 0.001:
        print(col)
        print(final_merge[col].isnull().sum()/len(final_merge[col]))
        cols.append(col)

new_final_merge = final_merge.drop(cols, axis=1)
new_final_merge.head()

gas_fiscal_year
0.04197627427975492
gas_fiscal_week
0.04197627427975492
gas_current_year_production
0.04197627427975492
gas_previous_year_production
0.04197627427975492
gas_current_year_cumulative_production
0.04197627427975492
gas_cumulative_difference
0.04197627427975492
VIX_returns_pct
0.3093468908877591
Volume
0.3093468908877591
DJA_volume_pct
0.3093468908877591
XLK_ret
0.311041585190979
XLI_ret
0.311041585190979
XLV_ret
0.311041585190979
XLE_ret
0.311041585190979
XLF_ret
0.311041585190979
CPI
0.30022161387042107
CPI_change_pct
0.30022161387042107


,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,KO,MCD,MMM,MRK,MSFT,NKE,PG,TRV,UNH,VZ,V,WBA,WMT,aerospace_funds_raised_mean,construction_funds_raised_mean,consumer_funds_raised_mean,crypto_funds_raised_mean,data_funds_raised_mean,education_funds_raised_mean,energy_funds_raised_mean,finance_funds_raised_mean,fitness_funds_raised_mean,food_funds_raised_mean,hardware_funds_raised_mean,healthcare_funds_raised_mean,hr_funds_raised_mean,infrastructure_funds_raised_mean,legal_funds_raised_mean,logistics_funds_raised_mean,manufacturing_funds_raised_mean,marketing_funds_raised_mean,media_funds_raised_mean,other_funds_raised_mean,product_funds_raised_mean,real estate_funds_raised_mean,recruiting_funds_raised_mean,retail_funds_raised_mean,sales_funds_raised_mean,security_funds_raised_mean,support_funds_raised_mean,transportation_funds_raised_mean,travel_funds_raised_mean,aerospace_funds_raised_sum,construction_funds_raised_sum,consumer_funds_raised_sum,crypto_funds_raised_sum,data_funds_raised_sum,education_funds_raised_sum,energy_funds_raised_sum,finance_funds_raised_sum,fitness_funds_raised_sum,food_funds_raised_sum,hardware_funds_raised_sum,healthcare_funds_raised_sum,hr_funds_raised_sum,infrastructure_funds_raised_sum,legal_funds_raised_sum,logistics_funds_raised_sum,manufacturing_funds_raised_sum,marketing_funds_raised_sum,media_funds_raised_sum,other_funds_raised_sum,product_funds_raised_sum,real estate_funds_raised_sum,recruiting_funds_raised_sum,retail_funds_raised_sum,sales_funds_raised_sum,security_funds_raised_sum,support_funds_raised_sum,transportation_funds_raised_sum,travel_funds_raised_sum,aerospace_laid_off_count_mean,construction_laid_off_count_mean,consumer_laid_off_count_mean,crypto_laid_off_count_mean,data_laid_off_count_mean,education_laid_off_count_mean,energy_laid_off_count_mean,finance_laid_off_count_mean,fitness_laid_off_count_mean,food_laid_off_count_mean,hardware_laid_off_count_mean,healthcare_laid_off_count_mean,hr_laid_off_count_mean,infrastructure_laid_off_count_mean,legal_laid_off_count_mean,logistics_laid_off_count_mean,manufacturing_laid_off_count_mean,marketing_laid_off_count_mean,media_laid_off_count_mean,other_laid_off_count_mean,product_laid_off_count_mean,real estate_laid_off_count_mean,recruiting_laid_off_count_mean,retail_laid_off_count_mean,sales_laid_off_count_mean,security_laid_off_count_mean,support_laid_off_count_mean,transportation_laid_off_count_mean,travel_laid_off_count_mean,aerospace_laid_off_count_sum,construction_laid_off_count_sum,consumer_laid_off_count_sum,crypto_laid_off_count_sum,data_laid_off_count_sum,education_laid_off_count_sum,energy_laid_off_count_sum,finance_laid_off_count_sum,fitness_laid_off_count_sum,food_laid_off_count_sum,hardware_laid_off_count_sum,healthcare_laid_off_count_sum,hr_laid_off_count_sum,infrastructure_laid_off_count_sum,legal_laid_off_count_sum,logistics_laid_off_count_sum,manufacturing_laid_off_count_sum,marketing_laid_off_count_sum,media_laid_off_count_sum,other_laid_off_count_sum,product_laid_off_count_sum,real estate_laid_off_count_sum,recruiting_laid_off_count_sum,retail_laid_off_count_sum,sales_laid_off_count_sum,security_laid_off_count_sum,support_laid_off_count_sum,transportation_laid_off_count_sum,travel_laid_off_count_sum,top_performing_sector_t+1
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-01 10:30:00,-0.073310,-0.133451,0.241968,0.942242,0.414206,-0.102929,-0.312257,0.810075,0.534720,0.232077,0.512068,-0.357439,0.251284,0.502541,0.258538,-0.204633,0.414504,0.476082,-0.049685,0.140398,-0.292642,-0.161615,0.086566,0.480305,0.232193,-0.081978,-0.242136,-0.177634,0.277781,0.471628,0.0,0.0,10.0,0.0,58.0,0.0,0.0,453.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1577.5,0.0,0.0,0.0,10.0,0.0,58.0,0.0,0.0,453.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [71]:
#get info about new dataframe
new_final_merge.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15342 entries, 2020-06-01 10:30:00 to 2023-06-01 16:00:00
Columns: 147 entries, AAPL to top_performing_sector_t+1
dtypes: float64(146), int64(1)
memory usage: 17.8 MB


In [72]:
#import libraries
%config inlinebackend.figure_format = 'retina'
import matplotlib.pyplot as plt
# Standard libraries
import os
import random
import warnings

# Data manipulation and analysis
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn import metrics
from sklearn.metrics import mean_squared_error

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Deep learning - TensorFlow/Keras
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers

from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.optimizers import SGD, Adam
from keras import losses
from keras.preprocessing import image

# Deep learning - PyTorch
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset, DataLoader, Subset

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [73]:
print(device)

cuda


## Simple Neural Net Classifier

In [74]:
class SimpleNNBatch_Clf(nn.Module):
    def __init__(self, input_size, output_size=9, dropout_prob=0.3):
        super(SimpleNNBatch_Clf, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.fc4 = nn.Linear(32, output_size)  # Output layer for 9 classes
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.fc4(x)
        #x = F.log_softmax(x, dim=1)  # Apply softmax to output layer
        return x



## Neural Network with residual connections

In [75]:
import torch.nn as nn
import torch.nn.functional as F

class Res_Clf(nn.Module):
    def __init__(self, input_size, output_size=9, dropout_prob=0.3):
        super(Res_Clf, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.relu1 = nn.LeakyReLU(negative_slope=0.01)

        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.relu2 = nn.LeakyReLU(negative_slope=0.01)
        self.fc2_residual = nn.Linear(128, 64)  # Residual connection for fc2

        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.relu3 = nn.LeakyReLU(negative_slope=0.01)
        self.fc3_residual = nn.Linear(64, 32)  # Residual connection for fc3

        self.fc4 = nn.Linear(32, output_size)  # Output layer for 9 classes
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        # Initial fully connected layer
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout(x)

        # First residual block
        identity = x  # Save the input to the block for the residual connection
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        identity = self.fc2_residual(identity)  # Transform the identity to match dimensions if needed
        x += identity  # Add the residual connection
        x = self.dropout(x)

        # Second residual block
        identity = x  # Save the input to the block for the residual connection
        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        identity = self.fc3_residual(identity)  # Transform the identity to match dimensions if needed
        x += identity  # Add the residual connection
        x = self.dropout(x)

        # Output layer
        x = self.fc4(x)
        return x


In [76]:
# custom function for reproducibility
def custom_set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [77]:
custom_set_seed(42)

## Using Dataset with removed columns because NN cannot handle NaNs

In [78]:
#label encode target column
le = LabelEncoder()
new_final_merge['top_performing_sector_t+1'] = le.fit_transform(new_final_merge['top_performing_sector_t+1'].astype(str))

new_final_merge['top_performing_sector_t+1'].unique()

array([5, 2, 1, 4, 8, 3, 6, 0, 7])

In [79]:
train_set = new_final_merge[:'2022-12-31']
test_set = new_final_merge['2023-01-01':]

print(train_set.shape)

(13214, 147)


In [80]:
X_train = train_set.drop(['top_performing_sector_t+1'], axis=1)
y_train = train_set['top_performing_sector_t+1']

X_test = test_set.drop(['top_performing_sector_t+1'], axis=1)
y_test = test_set['top_performing_sector_t+1']

#scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#convert to tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

#convert to tensor dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

#set batch size
batch_size = 32

#set train and test dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Training SimpleNN

In [82]:
input_size = X_train.shape[1]
output_size = 9  # For 9-class classification
PRINT_EVERY = 10

# Instantiate the model
simple_model = SimpleNNBatch_Clf(input_size=input_size, output_size=output_size, dropout_prob=0.2).to(device)

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multiclass classification
optimizer = optim.Adam(simple_model.parameters(), lr=0.01, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)  # Adjust step_size and gamma as needed

# Train the model
num_epochs = 250  # Adjust this based on convergence
train_losses = []
valid_losses = []

# Iterate over epochs
for epoch in range(num_epochs):
    simple_model.train()
    running_loss_train = 0
    running_loss_valid = 0
    for X_batch, y_batch in train_loader:  # Assuming all_loader is your DataLoader
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        output = simple_model(X_batch)
        loss = loss_fn(output, y_batch)  # y_batch is expected to be class indices
        loss.backward()
        optimizer.step()

        running_loss_train += loss.item() * X_batch.size(0)

    epoch_loss_train = running_loss_train / len(train_loader.dataset)
    train_losses.append(epoch_loss_train)

    # Adjust the learning rate
    scheduler.step()

    with torch.no_grad():

      simple_model.eval()

      for x_val, y_val in test_loader:
          x_val = x_val.to(device)
          y_val = y_val.to(device)
          y_hat = simple_model(x_val)
          loss = loss_fn(y_hat, y_val)

          running_loss_valid += loss.item() * x_val.size(0)

      epoch_loss_valid = running_loss_valid / len(test_loader.dataset)

      if epoch > 0 and epoch_loss_valid < min(valid_losses):
          best_epoch = epoch
          torch.save(simple_model.state_dict(), 'simple_model_checkpoint.pth')

      valid_losses.append(epoch_loss_valid)

    if epoch % PRINT_EVERY == 0:
      current_lr = scheduler.get_last_lr()[0]
      print(f"<{epoch}> - Train. loss: {epoch_loss_train:.10f} \t Valid. loss: {epoch_loss_valid:.10f} \t Learning Rate: {current_lr:10f}")

print(f'Lowest loss recorded in epoch: {best_epoch}')



<0> - Train. loss: 1.6222561861 	 Valid. loss: 1.8431179462 	 Learning Rate:   0.010000
<10> - Train. loss: 1.4657450783 	 Valid. loss: 1.9455838193 	 Learning Rate:   0.010000
<20> - Train. loss: 1.4618157953 	 Valid. loss: 1.8162842197 	 Learning Rate:   0.010000
<30> - Train. loss: 1.4603203127 	 Valid. loss: 1.6754048456 	 Learning Rate:   0.010000
<40> - Train. loss: 1.4534418780 	 Valid. loss: 1.6235441942 	 Learning Rate:   0.010000
<50> - Train. loss: 1.4335520797 	 Valid. loss: 1.6707609317 	 Learning Rate:   0.001000
<60> - Train. loss: 1.3937283687 	 Valid. loss: 1.6793373493 	 Learning Rate:   0.001000
<70> - Train. loss: 1.3829388042 	 Valid. loss: 1.7309204173 	 Learning Rate:   0.001000
<80> - Train. loss: 1.3862156374 	 Valid. loss: 1.6562827990 	 Learning Rate:   0.001000
<90> - Train. loss: 1.3717632040 	 Valid. loss: 1.7183899939 	 Learning Rate:   0.001000
<100> - Train. loss: 1.3712967168 	 Valid. loss: 1.7993178114 	 Learning Rate:   0.000100
<110> - Train. loss: 

In [84]:
# Load the best model (optional)
simple_model.load_state_dict(torch.load('simple_model_checkpoint.pth'))
simple_model.eval()  # Set the model to evaluation mode

all_preds = []
all_true = []

with torch.no_grad():
    for x_val, y_val in test_loader:
        x_val = x_val.to(device)
        # Forward pass
        outputs = simple_model(x_val)
        # Convert output logits to predicted class (argmax)
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_true.extend(y_val.cpu().numpy())

# Convert lists to numpy arrays if they are not already
all_preds = np.array(all_preds)
all_true = np.array(all_true)

In [85]:
from sklearn.metrics import classification_report
report = classification_report(all_true, all_preds)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        86
           1       0.00      0.00      0.00        86
           2       0.09      0.04      0.05       240
           3       0.00      0.00      0.00       106
           4       0.00      0.00      0.00       159
           5       0.00      0.00      0.00       106
           6       0.48      0.99      0.64       980
           7       0.00      0.00      0.00       165
           8       0.00      0.00      0.00       200

    accuracy                           0.46      2128
   macro avg       0.06      0.11      0.08      2128
weighted avg       0.23      0.46      0.30      2128



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Training Residual Neural Network

In [87]:
input_size = X_train.shape[1]
output_size = 9  # For 9-class classification
PRINT_EVERY = 10

# Instantiate the model
res_model = Res_Clf(input_size=input_size, output_size=output_size, dropout_prob=0.2).to(device)

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multiclass classification
optimizer = optim.Adam(res_model.parameters(), lr=0.01, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)  # Adjust step_size and gamma as needed

# Train the model
num_epochs = 250  # Adjust this based on convergence
train_losses = []
valid_losses = []

# Iterate over epochs
for epoch in range(num_epochs):
    res_model.train()
    running_loss_train = 0
    running_loss_valid = 0
    for X_batch, y_batch in train_loader:  # Assuming all_loader is your DataLoader
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        output = res_model(X_batch)
        loss = loss_fn(output, y_batch)  # y_batch is expected to be class indices
        loss.backward()
        optimizer.step()

        running_loss_train += loss.item() * X_batch.size(0)

    epoch_loss_train = running_loss_train / len(train_loader.dataset)
    train_losses.append(epoch_loss_train)

    # Adjust the learning rate
    scheduler.step()

    with torch.no_grad():

      res_model.eval()

      for x_val, y_val in test_loader:
          x_val = x_val.to(device)
          y_val = y_val.to(device)
          y_hat = res_model(x_val)
          loss = loss_fn(y_hat, y_val)

          running_loss_valid += loss.item() * x_val.size(0)

      epoch_loss_valid = running_loss_valid / len(test_loader.dataset)

      if epoch > 0 and epoch_loss_valid < min(valid_losses):
          best_epoch = epoch
          torch.save(res_model.state_dict(), 'res_model_checkpoint.pth')

      valid_losses.append(epoch_loss_valid)

    if epoch % PRINT_EVERY == 0:
      current_lr = scheduler.get_last_lr()[0]
      print(f"<{epoch}> - Train. loss: {epoch_loss_train:.10f} \t Valid. loss: {epoch_loss_valid:.10f} \t Learning Rate: {current_lr:10f}")

print(f'Lowest loss recorded in epoch: {best_epoch}')



<0> - Train. loss: 1.6779609181 	 Valid. loss: 1.8971505188 	 Learning Rate:   0.010000
<10> - Train. loss: 1.5120389200 	 Valid. loss: 2.3583607334 	 Learning Rate:   0.010000
<20> - Train. loss: 1.4939715707 	 Valid. loss: 1.8776633095 	 Learning Rate:   0.010000
<30> - Train. loss: 1.5003483742 	 Valid. loss: 1.8511034390 	 Learning Rate:   0.010000
<40> - Train. loss: 1.4993491305 	 Valid. loss: 1.9386510698 	 Learning Rate:   0.010000
<50> - Train. loss: 1.4603064167 	 Valid. loss: 2.2108145687 	 Learning Rate:   0.001000
<60> - Train. loss: 1.4010081663 	 Valid. loss: 2.3095573057 	 Learning Rate:   0.001000
<70> - Train. loss: 1.3748457897 	 Valid. loss: 2.3898948586 	 Learning Rate:   0.001000
<80> - Train. loss: 1.3683142014 	 Valid. loss: 2.2979974995 	 Learning Rate:   0.001000
<90> - Train. loss: 1.3529904942 	 Valid. loss: 2.4617529894 	 Learning Rate:   0.001000
<100> - Train. loss: 1.3381019526 	 Valid. loss: 2.6424899224 	 Learning Rate:   0.000100
<110> - Train. loss: 

In [88]:
# Load the best model (optional)
res_model.load_state_dict(torch.load('res_model_checkpoint.pth'))
res_model.eval()  # Set the model to evaluation mode

all_preds = []
all_true = []

with torch.no_grad():
    for x_val, y_val in test_loader:
        x_val = x_val.to(device)
        # Forward pass
        outputs = res_model(x_val)
        # Convert output logits to predicted class (argmax)
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_true.extend(y_val.cpu().numpy())

# Convert lists to numpy arrays if they are not already
all_preds = np.array(all_preds)
all_true = np.array(all_true)

In [89]:
from sklearn.metrics import classification_report
report = classification_report(all_true, all_preds)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        86
           1       0.00      0.00      0.00        86
           2       0.12      0.07      0.09       240
           3       0.00      0.00      0.00       106
           4       0.00      0.00      0.00       159
           5       0.00      0.00      0.00       106
           6       0.49      0.92      0.64       980
           7       0.09      0.05      0.06       165
           8       0.10      0.01      0.03       200

    accuracy                           0.44      2128
   macro avg       0.09      0.12      0.09      2128
weighted avg       0.25      0.44      0.31      2128



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Feature Selection

### Using xgboost to select features from the full dataset

In [90]:
train = final_merge[:'2022-12-31']
test = final_merge['2023-01-01':]

In [91]:
X_train_fs = train.drop(['top_performing_sector_t+1'], axis = 1)
y_train_fs = train['top_performing_sector_t+1']

X_test_fs = test.drop(['top_performing_sector_t+1'], axis=1)
y_test_fs = test['top_performing_sector_t+1']

# Specify parameters including the use of GPU
params = {'tree_method': 'hist', 'device': 'cuda'}
# Other XGBoost parameters can be included in 'params'

# Train the model
model = XGBClassifier(n_estimators = 300,learning_rate = 0.01, **params)
model.fit(X_train_fs, y_train_fs)

# Predict
predictions = model.predict(X_test_fs)

# Evaluate
accuracy = accuracy_score(y_test_fs, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

print(classification_report(y_test_fs, predictions))



Accuracy: 48.78%
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        86
           1       0.00      0.00      0.00        86
           2       0.20      0.59      0.29       240
           3       0.17      0.01      0.02       106
           4       0.17      0.03      0.05       159
           5       0.30      0.06      0.10       106
           6       0.71      0.87      0.78       980
           7       0.19      0.07      0.10       165
           8       0.25      0.11      0.15       200

    accuracy                           0.49      2128
   macro avg       0.22      0.19      0.17      2128
weighted avg       0.42      0.49      0.43      2128



In [92]:
importances = model.feature_importances_
# Assuming X_train is a DataFrame, get feature names
feature_names = X_train_fs.columns if isinstance(X_train_fs, pd.DataFrame) else range(X_train_fs.shape[1])

# Sort features by importance
importance_dict = dict(zip(feature_names, importances))
sorted_importance = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)


In [93]:
top_25_features = [feature for feature, _ in sorted_importance[:25]]
print(top_25_features)

['Volume', 'VIX_returns_pct', 'XLK_ret', 'DJA_volume_pct', 'DOW', 'XLF_ret', 'CVX', 'CPI_change_pct', 'XLE_ret', 'GS', 'legal_funds_raised_mean', 'security_laid_off_count_sum', 'travel_funds_raised_mean', 'healthcare_funds_raised_sum', 'JPM', 'marketing_laid_off_count_mean', 'gas_fiscal_year', 'NKE', 'CPI', 'infrastructure_funds_raised_mean', 'MRK', 'VZ', 'UNH', 'INTC', 'XLI_ret']


### Trying xgboost with the top 25 features

In [94]:
X_selected_train = X_train_fs[top_25_features]
X_selected_test = X_test_fs[top_25_features]

In [95]:
#fit xbooost with selected features
# Specify parameters including the use of GPU
params = {'tree_method': 'hist', 'device': 'cuda'}
# Other XGBoost parameters can be included in 'params'

# Train the model
model = XGBClassifier(n_estimators = 300,learning_rate = 0.01, **params)
model.fit(X_selected_train, y_train_fs)

# Predict
predictions = model.predict(X_selected_test)

# Evaluate
accuracy = accuracy_score(y_test_fs, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

print(classification_report(y_test_fs, predictions))


Accuracy: 48.21%
              precision    recall  f1-score   support

           0       1.00      0.01      0.02        86
           1       0.00      0.00      0.00        86
           2       0.19      0.57      0.28       240
           3       0.33      0.02      0.04       106
           4       0.11      0.04      0.06       159
           5       0.14      0.03      0.05       106
           6       0.74      0.85      0.79       980
           7       0.20      0.07      0.11       165
           8       0.25      0.15      0.19       200

    accuracy                           0.48      2128
   macro avg       0.33      0.19      0.17      2128
weighted avg       0.47      0.48      0.43      2128



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Tuning xgboost with randomsearchcv on the selected data

In [68]:
from scipy.stats import loguniform

# Define the XGBRegressor with the proper device specification
#to use without gpu remove tree_method='hist', device='cuda'
xgb_clf = XGBClassifier(tree_method='hist', device='cuda')

pipe_xgb = Pipeline([('xgb', xgb_clf)])
# Updated parameter grid with logarithmic scale and additional parameters
param_grid = {
    'xgb__n_estimators': [200, 300, 400, 500, 600],
    'xgb__max_depth': [2, 4, 6, 10, 15],
    'xgb__learning_rate': loguniform(0.01, 0.2), # Log scale
    'xgb__min_child_weight': [1, 2, 3],
    'xgb__reg_alpha': loguniform(1e-5, 1.0), # Log scale
    'xgb__reg_lambda': loguniform(1e-5, 1.0), # Log scale
    'xgb__gamma': [0.0, 1, 3, 6],
    'xgb__colsample_bytree': [0.4, 0.5, 0.7, 0.9],
    'xgb__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1]
}


rs_xgb = RandomizedSearchCV(pipe_xgb, param_grid, n_jobs=-1, random_state=42, n_iter=50, scoring='f1_macro')

# Fit model
rs_xgb.fit(X_selected_train, y_train_fs)

# Print best parameters and highest score
print('Best parameters found:\n', rs_xgb.best_params_)
print('Best F1 score:\n', rs_xgb.best_score_)
print('Best estimator:\n', rs_xgb.best_estimator_)

Best parameters found:
 {'xgb__colsample_bytree': 0.9, 'xgb__gamma': 1, 'xgb__learning_rate': 0.10342879811785984, 'xgb__max_depth': 10, 'xgb__min_child_weight': 3, 'xgb__n_estimators': 200, 'xgb__reg_alpha': 6.429002142212087e-05, 'xgb__reg_lambda': 0.31100755757274906, 'xgb__subsample': 0.8}
Best F1 score:
 0.14715447202720014
Best estimator:
 Pipeline(steps=[('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.9, device='cuda',
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=1, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.10342879811785984, max_bin=None

In [69]:
#save pck file
#save model parameters
import json
params = rs_xgb.best_params_
with open('xgb_model_params_fs.json', 'w') as fp:
    json.dump(params, fp)

#load model parameters
with open('xgb_model_params_fs.json', 'r') as fp:
    params = json.load(fp)

#set params
xgb_clf_fs = XGBClassifier(**params)



### Feature selection using the dataset where the columns that have NaNs are removed

In [96]:
train_set = new_final_merge[:'2022-12-31']
test_set = new_final_merge['2023-01-01':]

X_train = train_set.drop(['top_performing_sector_t+1'], axis=1)
y_train = train_set['top_performing_sector_t+1']

X_test = test_set.drop(['top_performing_sector_t+1'], axis=1)
y_test = test_set['top_performing_sector_t+1']


In [97]:
custom_set_seed(42)

In [98]:

# # Assuming X_train, X_test, y_train, y_test are your datasets
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test, label=y_test)

# Specify parameters including the use of GPU
params = {'tree_method': 'hist', 'device': 'cuda'}
# Other XGBoost parameters can be included in 'params'

# Train the
model = XGBClassifier(n_estimators = 300,learning_rate = 0.01, **params)
model.fit(X_train, y_train)

# Predict
predictions = model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(classification_report(y_test,predictions))


Accuracy: 48.50%
              precision    recall  f1-score   support

           0       0.08      0.01      0.02        86
           1       0.00      0.00      0.00        86
           2       0.19      0.47      0.27       240
           3       0.50      0.01      0.02       106
           4       0.21      0.06      0.09       159
           5       0.18      0.11      0.14       106
           6       0.69      0.87      0.77       980
           7       0.21      0.03      0.05       165
           8       0.28      0.19      0.23       200

    accuracy                           0.48      2128
   macro avg       0.26      0.19      0.18      2128
weighted avg       0.43      0.48      0.43      2128



In [99]:
importances = model.feature_importances_


In [100]:
# Assuming X_train is a DataFrame, get feature names
feature_names = X_train.columns if isinstance(X_train, pd.DataFrame) else range(X_train.shape[1])

# Sort features by importance
importance_dict = dict(zip(feature_names, importances))
sorted_importance = sorted(importance_dict.items(), key=lambda x: x[1], reverse=True)


In [101]:
top_25_features = [feature for feature, _ in sorted_importance[:25]]


In [102]:
top_25_features

['DOW',
 'CVX',
 'JNJ',
 'MSFT',
 'VZ',
 'BA',
 'consumer_funds_raised_sum',
 'NKE',
 'data_laid_off_count_sum',
 'JPM',
 'CRM',
 'support_laid_off_count_sum',
 'TRV',
 'consumer_laid_off_count_sum',
 'MCD',
 'AXP',
 'GS',
 'HD',
 'UNH',
 'transportation_laid_off_count_sum',
 'INTC',
 'CAT',
 'security_laid_off_count_sum',
 'product_funds_raised_sum',
 'DIS']

### Tuning xgboost with the top25 features from the cleaned up dataset (no NaNs)

In [103]:
# Assuming top_25_features_indices is a list of indices
X_train_selected = X_train[top_25_features]
X_test_selected = X_test[top_25_features]

y_train = train_set['top_performing_sector_t+1']
y_test = test_set['top_performing_sector_t+1']

In [79]:
from scipy.stats import loguniform

# Define the XGBRegressor with the proper device specification
#to use without gpu remove tree_method='hist', device='cuda'
xgb_clf = XGBClassifier(tree_method='hist', device='cuda')

pipe_xgb = Pipeline([('xgb', xgb_clf)])
# Updated parameter grid with logarithmic scale and additional parameters
param_grid = {
    'xgb__n_estimators': [200, 300, 400, 500, 600],
    'xgb__max_depth': [2, 4, 6, 10, 15],
    'xgb__learning_rate': loguniform(0.01, 0.2), # Log scale
    'xgb__min_child_weight': [1, 2, 3],
    'xgb__reg_alpha': loguniform(1e-5, 1.0), # Log scale
    'xgb__reg_lambda': loguniform(1e-5, 1.0), # Log scale
    'xgb__gamma': [0.0, 1, 3, 6],
    'xgb__colsample_bytree': [0.4, 0.5, 0.7, 0.9],
    'xgb__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1]
}


rs_xgb = RandomizedSearchCV(pipe_xgb, param_grid, n_jobs=-1, random_state=42, n_iter=50, scoring='f1_macro')

# Fit model
rs_xgb.fit(X_train_selected, y_train)

# Print best parameters and highest score
print('Best parameters found:\n', rs_xgb.best_params_)
print('Best F1 score:\n', rs_xgb.best_score_)
print('Best estimator:\n', rs_xgb.best_estimator_)

Best parameters found:
 {'xgb__colsample_bytree': 0.9, 'xgb__gamma': 3, 'xgb__learning_rate': 0.1941287044914516, 'xgb__max_depth': 4, 'xgb__min_child_weight': 3, 'xgb__n_estimators': 400, 'xgb__reg_alpha': 0.0002995387010189221, 'xgb__reg_lambda': 0.0004099716456444887, 'xgb__subsample': 0.5}
Best F1 score:
 0.18171752800334603
Best estimator:
 Pipeline(steps=[('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.9, device='cuda',
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=3, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.1941287044914516, max_bin=None,

In [80]:
#save pck file
#save model parameters
import json
params = rs_xgb.best_params_
with open('xgb_model_params_fs_selected.json', 'w') as fp:
    json.dump(params, fp)

#load model parameters
with open('xgb_model_params_fs_selected.json', 'r') as fp:
    params = json.load(fp)

#set params
xgb_clf_fs = XGBClassifier(**params)

#fit the model
xgb_clf.fit(X_train_selected, y_train)

# Generate predictions on the test set
y_test_predictions = xgb_clf.predict(X_test_selected)

# Evaluate the predictions

accuracy = accuracy_score(y_test, y_test_predictions)
print("Test Accuracy:", accuracy)
print(classification_report(y_test, y_test_predictions))

### Using the top 25 features from the cleaned dataset we try the neural net

In [104]:
# Assuming top_25_features_indices is a list of indices
X_train_selected = X_train[top_25_features]
X_test_selected = X_test[top_25_features]

y_train = train_set['top_performing_sector_t+1']
y_test = test_set['top_performing_sector_t+1']
#scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_selected)
X_test = scaler.transform(X_test_selected)

#convert to tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

#convert to tensor dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

#set batch size
batch_size = 32

#set train and test dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Training the Residual Network with selected features

In [105]:
input_size = X_train.shape[1]
output_size = 9  # For 9-class classification
PRINT_EVERY = 10

# Instantiate the model
model = Res_Clf(input_size=input_size, output_size=output_size, dropout_prob=0.2).to(device)

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)  # Adjust step_size and gamma as needed

# Train the model
num_epochs = 250  # Adjust this based on convergence
train_losses = []
valid_losses = []

# Iterate over epochs
for epoch in range(num_epochs):
    model.train()
    running_loss_train = 0
    running_loss_valid = 0
    for X_batch, y_batch in train_loader:  # Assuming all_loader is your DataLoader
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = loss_fn(output, y_batch)  # y_batch is expected to be class indices
        loss.backward()
        optimizer.step()

        running_loss_train += loss.item() * X_batch.size(0)

    epoch_loss_train = running_loss_train / len(train_loader.dataset)
    train_losses.append(epoch_loss_train)

    # Adjust the learning rate
    scheduler.step()

    with torch.no_grad():

      model.eval()

      for x_val, y_val in test_loader:
          x_val = x_val.to(device)
          y_val = y_val.to(device)
          y_hat = model(x_val)
          loss = loss_fn(y_hat, y_val)

          running_loss_valid += loss.item() * x_val.size(0)

      epoch_loss_valid = running_loss_valid / len(test_loader.dataset)

      if epoch > 0 and epoch_loss_valid < min(valid_losses):
          best_epoch = epoch
          torch.save(model.state_dict(), 'res_select_checkpoint.pth')

      valid_losses.append(epoch_loss_valid)

    if epoch % PRINT_EVERY == 0:
      current_lr = scheduler.get_last_lr()[0]
      print(f"<{epoch}> - Train. loss: {epoch_loss_train:.10f} \t Valid. loss: {epoch_loss_valid:.10f} \t Learning Rate: {current_lr:10f}")

print(f'Lowest loss recorded in epoch: {best_epoch}')



<0> - Train. loss: 1.5719667339 	 Valid. loss: 1.5463215742 	 Learning Rate:   0.010000
<10> - Train. loss: 1.4813559119 	 Valid. loss: 1.4826700775 	 Learning Rate:   0.010000
<20> - Train. loss: 1.4694509190 	 Valid. loss: 1.4785911008 	 Learning Rate:   0.010000
<30> - Train. loss: 1.4724405903 	 Valid. loss: 1.5043461786 	 Learning Rate:   0.010000
<40> - Train. loss: 1.4836431144 	 Valid. loss: 1.5210200057 	 Learning Rate:   0.010000
<50> - Train. loss: 1.4401935619 	 Valid. loss: 1.5391813509 	 Learning Rate:   0.001000
<60> - Train. loss: 1.4041296713 	 Valid. loss: 1.4689010216 	 Learning Rate:   0.001000
<70> - Train. loss: 1.3883272107 	 Valid. loss: 1.4402394901 	 Learning Rate:   0.001000
<80> - Train. loss: 1.3796896289 	 Valid. loss: 1.6079902067 	 Learning Rate:   0.001000
<90> - Train. loss: 1.3688392472 	 Valid. loss: 1.4096919728 	 Learning Rate:   0.001000
<100> - Train. loss: 1.3621549230 	 Valid. loss: 1.5304001946 	 Learning Rate:   0.000100
<110> - Train. loss: 

No learning happening with the selected features on the residual network so no evaluation needed

### Training the Simple NN on selected features

In [106]:
input_size = X_train.shape[1]
output_size = 9  # For 9-class classification
PRINT_EVERY = 10

# Instantiate the model
model = SimpleNNBatch_Clf(input_size=input_size, output_size=output_size, dropout_prob=0.2).to(device)

# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multiclass classification
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)  # Adjust step_size and gamma as needed

# Train the model
num_epochs = 250  # Adjust this based on convergence
train_losses = []
valid_losses = []

# Iterate over epochs
for epoch in range(num_epochs):
    model.train()
    running_loss_train = 0
    running_loss_valid = 0
    for X_batch, y_batch in train_loader:  # Assuming all_loader is your DataLoader
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = loss_fn(output, y_batch)  # y_batch is expected to be class indices
        loss.backward()
        optimizer.step()

        running_loss_train += loss.item() * X_batch.size(0)

    epoch_loss_train = running_loss_train / len(train_loader.dataset)
    train_losses.append(epoch_loss_train)

    # Adjust the learning rate
    scheduler.step()

    with torch.no_grad():

      model.eval()

      for x_val, y_val in test_loader:
          x_val = x_val.to(device)
          y_val = y_val.to(device)
          y_hat = model(x_val)
          loss = loss_fn(y_hat, y_val)

          running_loss_valid += loss.item() * x_val.size(0)

      epoch_loss_valid = running_loss_valid / len(test_loader.dataset)

      if epoch > 0 and epoch_loss_valid < min(valid_losses):
          best_epoch = epoch
          torch.save(model.state_dict(), 'mlp_proj_checkpoint_base.pth')

      valid_losses.append(epoch_loss_valid)

    if epoch % PRINT_EVERY == 0:
      current_lr = scheduler.get_last_lr()[0]
      print(f"<{epoch}> - Train. loss: {epoch_loss_train:.10f} \t Valid. loss: {epoch_loss_valid:.10f} \t Learning Rate: {current_lr:10f}")

print(f'Lowest loss recorded in epoch: {best_epoch}')



<0> - Train. loss: 1.5223460503 	 Valid. loss: 1.5507431427 	 Learning Rate:   0.010000
<10> - Train. loss: 1.4345554762 	 Valid. loss: 1.4682286196 	 Learning Rate:   0.010000
<20> - Train. loss: 1.4281585696 	 Valid. loss: 1.5403691374 	 Learning Rate:   0.010000
<30> - Train. loss: 1.4285186018 	 Valid. loss: 1.4917016834 	 Learning Rate:   0.010000
<40> - Train. loss: 1.4226865322 	 Valid. loss: 1.4943823101 	 Learning Rate:   0.010000
<50> - Train. loss: 1.4188701783 	 Valid. loss: 1.4312517815 	 Learning Rate:   0.001000
<60> - Train. loss: 1.3847541754 	 Valid. loss: 1.4205094355 	 Learning Rate:   0.001000
<70> - Train. loss: 1.3839833640 	 Valid. loss: 1.4282098143 	 Learning Rate:   0.001000
<80> - Train. loss: 1.3747361881 	 Valid. loss: 1.4132908077 	 Learning Rate:   0.001000
<90> - Train. loss: 1.3779270053 	 Valid. loss: 1.4119655838 	 Learning Rate:   0.001000
<100> - Train. loss: 1.3690952514 	 Valid. loss: 1.4195409099 	 Learning Rate:   0.000100
<110> - Train. loss: 

In [107]:
from sklearn.metrics import f1_score

# Load the best model (optional)
model.load_state_dict(torch.load('mlp_proj_checkpoint_base.pth'))
model.eval()  # Set the model to evaluation mode

all_preds = []
all_true = []

with torch.no_grad():
    for x_val, y_val in test_loader:
        x_val = x_val.to(device)
        # Forward pass
        outputs = model(x_val)
        # Convert output logits to predicted class (argmax)
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_true.extend(y_val.cpu().numpy())

# Convert lists to numpy arrays if they are not already
all_preds = np.array(all_preds)
all_true = np.array(all_true)


In [108]:
from sklearn.metrics import classification_report
report = classification_report(all_true, all_preds)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [109]:
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        86
           1       0.00      0.00      0.00        86
           2       0.18      0.50      0.27       240
           3       0.00      0.00      0.00       106
           4       0.00      0.00      0.00       159
           5       0.00      0.00      0.00       106
           6       0.62      0.90      0.73       980
           7       0.00      0.00      0.00       165
           8       0.33      0.07      0.12       200

    accuracy                           0.48      2128
   macro avg       0.13      0.16      0.12      2128
weighted avg       0.34      0.48      0.38      2128



# Sequential Neural Networks

This section shows performance of LSTM, GRU, and a combined metal model using LSTM and GRU with xgboost then with logistic regression. The data here is the cleaned data (new_final_merge) that has the columns with Nans removed. Here we use a train,val, test split because of the Meta Model Approach.

## LSTM

In [110]:
import numpy as np

def create_sequences(features, target, seq_length):
    xs = []
    ys = []
    for i in range(len(features) - seq_length):
        seq = features[i:i + seq_length]
        label = target[i + seq_length]  # Align with the target just after the end of the sequence
        xs.append(seq)
        ys.append(label)
    return np.array(xs), np.array(ys)


In [111]:
train_set = new_final_merge[:'2022-6-30']
val_set = new_final_merge['2022-7-1':'2022-12-31']
test_set = new_final_merge['2023-01-01':]

X_train = train_set.drop(['top_performing_sector_t+1'], axis=1)
X_val = val_set.drop(['top_performing_sector_t+1'], axis=1)

y_train = train_set['top_performing_sector_t+1']
y_val = val_set['top_performing_sector_t+1']

X_test = test_set.drop(['top_performing_sector_t+1'], axis=1)
y_test = test_set['top_performing_sector_t+1']

In [112]:
seq_length = 5  # This is the number of time steps in each input sequence
feature_data = X_train.to_numpy()
target_data = y_train.to_numpy()
X, y = create_sequences(feature_data, target_data, seq_length)
X_test,y_test = create_sequences(X_test,y_test,seq_length)
X_val, y_val = create_sequences(X_val,y_val,seq_length)


In [113]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq, input_lengths):
        # Pack the input sequence
        packed_input = rnn_utils.pack_padded_sequence(input_seq, input_lengths, batch_first=True, enforce_sorted=False)

        # Forward pass through LSTM
        packed_output, (hidden, cell) = self.lstm(packed_input)

        # Unpack the output
        output, _ = rnn_utils.pad_packed_sequence(packed_output, batch_first=True)

        # Get the last timestep output for each sequence in the batch
        last_outputs = output[torch.arange(output.size(0)), input_lengths - 1]

        # Pass the last output of each sequence to the linear layer
        predictions = self.linear(last_outputs)
        return predictions


In [114]:
import torch

# Convert your sequences and labels to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)  # Features should be float
y_tensor = torch.tensor(y, dtype=torch.long)  # Labels for classification should be long
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)  # Features should be float
y_test_tensor = torch.tensor(y_test, dtype=torch.long)  # Labels for classification should be long
X_val_tensor = torch.tensor(X_val,dtype=torch.float32)
y_val_tensor = torch.tensor(y_val,dtype=torch.long)


In [115]:
class CustomDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        label = self.labels[idx]
        seq_length = len(sequence)  # Adjust this according to how your sequences are represented
        return sequence, seq_length, label


In [116]:
# sequences and labels are your data
sequences = X_tensor
labels = y_tensor

# Create the dataset and data loader
dataset = CustomDataset(sequences, labels)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

val_dataset = CustomDataset(X_val_tensor,y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [117]:
# Assuming X is a tensor or a NumPy array with shape (num_samples, seq_length, num_features)
input_size = X.shape[-1]  # The last dimension is typically the number of features

# Now use this input_size to define your model
hidden_layer_size = 50  # Hidden layer size of the LSTM
output_size = 9  # For 9-class classification
num_layers = 1  # Number of LSTM layers

lstm_model = LSTMModel(input_size, hidden_layer_size, output_size, num_layers).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.01, weight_decay=1e-4)  # Learning rate can be adjusted
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)


In [118]:
num_epochs = 150
for epoch in range(num_epochs):
    lstm_model.train()
    total_loss = 0

    for X_batch, seq_lengths, y_batch in data_loader:
        X_batch, seq_lengths, y_batch = X_batch.to(device), seq_lengths.to(device), y_batch.to(device)

        optimizer.zero_grad()
        predictions = lstm_model(X_batch, seq_lengths.cpu())
        loss = loss_function(predictions, y_batch)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(data_loader)}, LR: {current_lr}')


Epoch 1/150, Loss: 1.387895542938072, LR: 0.01
Epoch 2/150, Loss: 1.0159059466721416, LR: 0.01
Epoch 3/150, Loss: 0.923372287829001, LR: 0.01
Epoch 4/150, Loss: 0.8857259359266665, LR: 0.01
Epoch 5/150, Loss: 0.8634915887056528, LR: 0.01
Epoch 6/150, Loss: 0.8482476507758235, LR: 0.01
Epoch 7/150, Loss: 0.8258030157726448, LR: 0.01
Epoch 8/150, Loss: 0.814472836059135, LR: 0.01
Epoch 9/150, Loss: 0.8086266988390559, LR: 0.01
Epoch 10/150, Loss: 0.7876670052697351, LR: 0.01
Epoch 11/150, Loss: 0.7708081585150939, LR: 0.01
Epoch 12/150, Loss: 0.7782832710771589, LR: 0.01
Epoch 13/150, Loss: 0.7634671390414596, LR: 0.01
Epoch 14/150, Loss: 0.7622738461773675, LR: 0.01
Epoch 15/150, Loss: 0.7602272385412508, LR: 0.01
Epoch 16/150, Loss: 0.7470743648103766, LR: 0.01
Epoch 17/150, Loss: 0.752671914102437, LR: 0.01
Epoch 18/150, Loss: 0.7340113829385053, LR: 0.01
Epoch 19/150, Loss: 0.7457307919904634, LR: 0.01
Epoch 20/150, Loss: 0.7483962174202945, LR: 0.01
Epoch 21/150, Loss: 0.73772235935

In [119]:
lstm_model.eval()  # Set the model to evaluation mode
predictions = []
actuals = []

with torch.no_grad():
    for X_batch,seq_lengths, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_test_pred = lstm_model(X_batch, seq_lengths.cpu())
        _, predicted_labels = torch.max(y_test_pred, 1)
        predictions.extend(predicted_labels.view(-1).cpu())
        actuals.extend(y_batch.view(-1).cpu())

# Calculate metrics here
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(actuals, predictions)
print(f'Accuracy: {accuracy}')
print(classification_report(actuals, predictions))


Accuracy: 0.36834667922750824
              precision    recall  f1-score   support

           0       0.12      0.12      0.12        86
           1       0.14      0.08      0.10        86
           2       0.18      0.22      0.19       240
           3       0.14      0.13      0.14       106
           4       0.21      0.14      0.17       159
           5       0.26      0.10      0.15       106
           6       0.54      0.63      0.58       975
           7       0.14      0.16      0.15       165
           8       0.21      0.14      0.17       200

    accuracy                           0.37      2123
   macro avg       0.22      0.19      0.20      2123
weighted avg       0.35      0.37      0.35      2123



## GRU

In [120]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size, num_layers=1):
        super(GRUModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.gru = nn.GRU(input_size, hidden_layer_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq, input_lengths):
        # Pack the input sequence
        packed_input = rnn_utils.pack_padded_sequence(input_seq, input_lengths, batch_first=True, enforce_sorted=False)

        # Forward pass through GRU
        packed_output, hidden = self.gru(packed_input)

        # Unpack the output
        output, _ = rnn_utils.pad_packed_sequence(packed_output, batch_first=True)

        # Get the last timestep output for each sequence in the batch
        last_outputs = output[torch.arange(output.size(0)), input_lengths - 1]

        # Pass the last output of each sequence to the linear layer
        predictions = self.linear(last_outputs)
        return predictions


In [121]:
# Instantiate the model
input_size = X.shape[-1]  # Number of input features
hidden_layer_size = 50
output_size = 9   # Number of classes for classification
num_layers = 1

gru_model = GRUModel(input_size, hidden_layer_size, output_size, num_layers)
gru_model.to(device)  # Move the model to the appropriate device (GPU or CPU)

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru_model.parameters(), lr=0.01, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

# Training loop
num_epochs = 150
for epoch in range(num_epochs):
    gru_model.train()  # Set the model to training mode
    total_loss = 0

    for X_batch, seq_lengths, y_batch in data_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        seq_lengths = seq_lengths.cpu()  # Sequence lengths should be on CPU

        optimizer.zero_grad()

        predictions = gru_model(X_batch, seq_lengths)
        loss = loss_function(predictions, y_batch)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(data_loader)}, LR: {current_lr}')


Epoch 1/150, Loss: 1.3132008251485165, LR: 0.01
Epoch 2/150, Loss: 0.9737431846402429, LR: 0.01
Epoch 3/150, Loss: 0.9199052943481697, LR: 0.01
Epoch 4/150, Loss: 0.8861150635971321, LR: 0.01
Epoch 5/150, Loss: 0.8647726085092928, LR: 0.01
Epoch 6/150, Loss: 0.8448990414271483, LR: 0.01
Epoch 7/150, Loss: 0.84917536879087, LR: 0.01
Epoch 8/150, Loss: 0.8450372749799723, LR: 0.01
Epoch 9/150, Loss: 0.8402886967938226, LR: 0.01
Epoch 10/150, Loss: 0.823939587887343, LR: 0.01
Epoch 11/150, Loss: 0.8269850771885376, LR: 0.01
Epoch 12/150, Loss: 0.8322074943118625, LR: 0.01
Epoch 13/150, Loss: 0.8308552792659393, LR: 0.01
Epoch 14/150, Loss: 0.8214281587092368, LR: 0.01
Epoch 15/150, Loss: 0.814293626818929, LR: 0.01
Epoch 16/150, Loss: 0.8149291333762018, LR: 0.01
Epoch 17/150, Loss: 0.8053138425221315, LR: 0.01
Epoch 18/150, Loss: 0.7952137723251864, LR: 0.01
Epoch 19/150, Loss: 0.7881352730520494, LR: 0.01
Epoch 20/150, Loss: 0.7875939791982954, LR: 0.01
Epoch 21/150, Loss: 0.79298087166

In [122]:
gru_model.eval()  # Set the model to evaluation mode
predictions = []
actuals = []

with torch.no_grad():
    for X_batch,seq_lengths, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_test_pred = gru_model(X_batch, seq_lengths.cpu())
        _, predicted_labels = torch.max(y_test_pred, 1)
        predictions.extend(predicted_labels.view(-1).cpu())
        actuals.extend(y_batch.view(-1).cpu())

# Calculate metrics here
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(actuals, predictions)
print(f'Accuracy: {accuracy}')
print(classification_report(actuals, predictions))


Accuracy: 0.37447008949599625
              precision    recall  f1-score   support

           0       0.19      0.19      0.19        86
           1       0.09      0.13      0.11        86
           2       0.22      0.24      0.23       240
           3       0.32      0.16      0.21       106
           4       0.17      0.18      0.18       159
           5       0.08      0.09      0.09       106
           6       0.58      0.62      0.60       975
           7       0.21      0.12      0.15       165
           8       0.16      0.14      0.15       200

    accuracy                           0.37      2123
   macro avg       0.22      0.21      0.21      2123
weighted avg       0.37      0.37      0.37      2123



## Meta Model

In [123]:
lstm_predictions = []
gru_predictions = []
all_labels = []

with torch.no_grad():
    for X_batch, seq_lengths, y_batch in val_loader:
        X_batch = X_batch.to(device)
        seq_lengths = seq_lengths.cpu()

        lstm_pred = lstm_model(X_batch, seq_lengths)
        gru_pred = gru_model(X_batch, seq_lengths)

        lstm_predictions.append(lstm_pred.cpu())
        gru_predictions.append(gru_pred.cpu())
        all_labels.append(y_batch.cpu())

# Concatenate all predictions and labels
lstm_combined = torch.cat(lstm_predictions, dim=0)
gru_combined = torch.cat(gru_predictions, dim=0)
y_combined = torch.cat(all_labels, dim=0)

# Verify shapes
print("Combined LSTM predictions shape:", lstm_combined.shape)
print("Combined GRU predictions shape:", gru_combined.shape)
print("Combined labels shape:", y_combined.shape)


Combined LSTM predictions shape: torch.Size([2571, 9])
Combined GRU predictions shape: torch.Size([2571, 9])
Combined labels shape: torch.Size([2571])


In [124]:
X_combined = torch.cat((lstm_combined, gru_combined), dim=1).numpy()
print("Shape of combined features for meta model:", X_combined.shape)  # Should be [2571, 18]


Shape of combined features for meta model: (2571, 18)


In [125]:
y_combined_np = y_combined.numpy()
print("Shape of combined labels for meta model:", y_combined_np.shape)  # Should be [2571]

Shape of combined labels for meta model: (2571,)


In [126]:
import xgboost as xgb

# Create the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(objective='multi:softprob', num_class=9, tree_method = "hist", device = 'cuda',n_estimators = 400)

# Train the classifier
xgb_classifier.fit(X_combined, y_combined_np)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None, num_class=9,
              num_parallel_tree=None, ...)

In [127]:
lstm_test_predictions = []
gru_test_predictions = []
all_test_labels = []

with torch.no_grad():
    for X_batch, seq_lengths, y_batch in test_loader:
        X_batch = X_batch.to(device)
        seq_lengths = seq_lengths.cpu()

        lstm_pred = lstm_model(X_batch, seq_lengths)
        gru_pred = gru_model(X_batch, seq_lengths)

        lstm_test_predictions.append(lstm_pred.cpu())
        gru_test_predictions.append(gru_pred.cpu())
        all_test_labels.append(y_batch.cpu())

# Concatenate all predictions and labels
lstm_test_combined = torch.cat(lstm_test_predictions, dim=0)
gru_test_combined = torch.cat(gru_test_predictions, dim=0)
y_test_combined = torch.cat(all_test_labels, dim=0)

# Verify shapes
print("Combined LSTM predictions shape:", lstm_test_combined.shape)
print("Combined GRU predictions shape:", gru_test_combined.shape)
print("Combined labels shape:", y_test_combined.shape)


Combined LSTM predictions shape: torch.Size([2123, 9])
Combined GRU predictions shape: torch.Size([2123, 9])
Combined labels shape: torch.Size([2123])


In [128]:
X_test_combined = torch.cat((lstm_test_combined, gru_test_combined), dim=1).numpy()


In [129]:
test_meta_predictions = xgb_classifier.predict(X_test_combined)


In [130]:
y_test_np = y_test_combined.numpy()
# Evaluate the predictions
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test_np, test_meta_predictions)
print("Test Accuracy:", accuracy)
print(classification_report(y_test_np, test_meta_predictions))


Test Accuracy: 0.4479510127178521
              precision    recall  f1-score   support

           0       0.24      0.09      0.13        86
           1       0.20      0.09      0.13        86
           2       0.24      0.20      0.22       240
           3       0.52      0.12      0.20       106
           4       0.27      0.14      0.18       159
           5       0.13      0.05      0.07       106
           6       0.51      0.83      0.64       975
           7       0.31      0.10      0.15       165
           8       0.27      0.09      0.13       200

    accuracy                           0.45      2123
   macro avg       0.30      0.19      0.20      2123
weighted avg       0.38      0.45      0.38      2123



## Tuned XGBoost as Meta Model

In [137]:
from scipy.stats import loguniform

xgb_clf = xgb.XGBClassifier(objective='multi:softprob', num_class=9, tree_method = "hist", device = 'cuda')

pipe_xgb = Pipeline([('xgb', xgb_clf)])
# Updated parameter grid with logarithmic scale and additional parameters
param_grid = {
    'xgb__n_estimators': [200, 300, 400, 500, 600],
    'xgb__max_depth': [2, 4, 6, 10, 15],
    'xgb__learning_rate': loguniform(0.01, 0.2), # Log scale
    'xgb__min_child_weight': [1, 2, 3],
    'xgb__reg_alpha': loguniform(1e-5, 1.0), # Log scale
    'xgb__reg_lambda': loguniform(1e-5, 1.0), # Log scale
    'xgb__gamma': [0.0, 1, 3, 6],
    'xgb__colsample_bytree': [0.4, 0.5, 0.7, 0.9],
    'xgb__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1]
}


rs_xgb = RandomizedSearchCV(pipe_xgb, param_grid, n_jobs=-1, random_state=42, n_iter=50, scoring='f1_macro')

# Fit model
rs_xgb.fit(X_combined, y_combined_np)

# Print best parameters and highest score
print('Best parameters found:\n', rs_xgb.best_params_)
print('Best F1 score:\n', rs_xgb.best_score_)
print('Best estimator:\n', rs_xgb.best_estimator_)

Best parameters found:
 {'xgb__colsample_bytree': 0.5, 'xgb__gamma': 0.0, 'xgb__learning_rate': 0.0351489428352735, 'xgb__max_depth': 10, 'xgb__min_child_weight': 2, 'xgb__n_estimators': 400, 'xgb__reg_alpha': 0.06155035361040185, 'xgb__reg_lambda': 1.1802321563258188e-05, 'xgb__subsample': 1}
Best F1 score:
 0.2621968342315045
Best estimator:
 Pipeline(steps=[('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.5, device='cuda',
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=0.0, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.0351489428352735, max_bin=None

In [138]:
#Fit the best params on all the train data
params = rs_xgb.best_params_
xgb_clf = xgb.XGBClassifier(**params)
xgb_clf.fit(X_combined, y_combined_np)

# Generate predictions on the test set
y_test_predictions = xgb_clf.predict(X_test_combined)

# Evaluate the predictions

accuracy = accuracy_score(y_test_np, y_test_predictions)
print("Test Accuracy:", accuracy)
print(classification_report(y_test_np, y_test_predictions))

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:51:57] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "xgb__colsample_bytree", "xgb__gamma", "xgb__learning_rate", "xgb__max_depth", "xgb__min_child_weight", "xgb__n_estimators", "xgb__reg_alpha", "xgb__reg_lambda", "xgb__subsample" } are not used.

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.4418276024493641
              precision    recall  f1-score   support

           0       0.38      0.10      0.16        86
           1       0.13      0.08      0.10        86
           2       0.22      0.18      0.20       240
           3       0.45      0.09      0.16       106
           4       0.34      0.12      0.18       159
           5       0.23      0.07      0.10       106
           6       0.50      0.84      0.63       975
           7       0.30      0.08      0.12       165
           8       0.18      0.07      0.10       200

    accuracy                           0.44      2123
   macro avg       0.30      0.18      0.19      2123
weighted avg       0.38      0.44      0.37      2123



In [139]:
#Fitting best Estimator
xgb_best = rs_xgb.best_estimator_

# Generate predictions on the test set
y_test_predictions = xgb_best.predict(X_test_combined)

# Evaluate the predictions

accuracy = accuracy_score(y_test_np, y_test_predictions)
print("Test Accuracy:", accuracy)
print(classification_report(y_test_np, y_test_predictions))

Test Accuracy: 0.4686764013188884
              precision    recall  f1-score   support

           0       0.33      0.07      0.12        86
           1       0.23      0.08      0.12        86
           2       0.22      0.18      0.20       240
           3       0.53      0.09      0.16       106
           4       0.42      0.13      0.19       159
           5       0.42      0.05      0.08       106
           6       0.50      0.90      0.64       975
           7       0.50      0.08      0.15       165
           8       0.39      0.09      0.14       200

    accuracy                           0.47      2123
   macro avg       0.39      0.18      0.20      2123
weighted avg       0.43      0.47      0.38      2123



## Simpler Classifier as Meta Model

Using logistic regression as a simpler meta model instead of xgboost

In [131]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()
X_combined_scaled = scaler.fit_transform(X_combined)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_combined_scaled, y_combined_np)


LogisticRegression(max_iter=1000)

In [132]:
# Generate predictions on the test set
X_test_combined_scaled = scaler.transform(X_test_combined)
y_pred = log_reg.predict(X_test_combined_scaled)  # Make sure X_test_combined is prepared similarly

# Evaluate the model
accuracy = accuracy_score(y_test_np, y_pred)
print("Test Accuracy:", accuracy)
print(classification_report(y_test_np, y_pred))

Test Accuracy: 0.47197362223268957
              precision    recall  f1-score   support

           0       0.14      0.09      0.11        86
           1       0.26      0.08      0.12        86
           2       0.30      0.12      0.17       240
           3       0.83      0.09      0.17       106
           4       0.35      0.13      0.19       159
           5       0.75      0.03      0.05       106
           6       0.50      0.93      0.65       975
           7       0.35      0.05      0.09       165
           8       0.35      0.08      0.13       200

    accuracy                           0.47      2123
   macro avg       0.43      0.18      0.19      2123
weighted avg       0.44      0.47      0.37      2123



basic logistic regression not so good so we then use randomsearch cv to tune

In [134]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Scale the features
scaler = StandardScaler()
X_combined_scaled = scaler.fit_transform(X_combined)

# Set up the logistic regression model with hyperparameter tuning
log_reg = LogisticRegression(max_iter=1000)
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'penalty': ['l2', 'none']  # Regularization type
}

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(log_reg, param_grid, cv=5, scoring='f1_macro')  # Adjust scoring method based on your goals
grid_search.fit(X_combined_scaled, y_combined_np)

# Best model after grid search
best_model = grid_search.best_estimator_

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour

In [135]:
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have a test dataset prepared similarly
X_test_combined_scaled = scaler.transform(X_test_combined)  # Scale the test data features
y_test_predictions = best_model.predict(X_test_combined_scaled)

# Evaluate the model
test_accuracy = accuracy_score(y_test_np, y_test_predictions)
print("Test Accuracy:", test_accuracy)
print(classification_report(y_test_np, y_test_predictions))


Test Accuracy: 0.47244465379180406
              precision    recall  f1-score   support

           0       0.15      0.09      0.12        86
           1       0.24      0.08      0.12        86
           2       0.33      0.12      0.17       240
           3       0.37      0.10      0.16       106
           4       0.43      0.13      0.19       159
           5       0.75      0.03      0.05       106
           6       0.50      0.92      0.65       975
           7       0.38      0.05      0.10       165
           8       0.28      0.08      0.12       200

    accuracy                           0.47      2123
   macro avg       0.38      0.18      0.19      2123
weighted avg       0.43      0.47      0.37      2123



## Summary